### Machine Learning with the Equipment Failure Prediction. 

<h3 style="color:#01308e;">Overview</h3>

**Now I will take the selected/pre-processed/imputed datasets and feed this into different ML models by using cross validation.**

<h3 style="color:#01308e;">Business Problem in machine learning terms:</h3>

**Given the data points with their respective features, use classification to find out whether the data points belong to surface failure or downhole failure.**

<h3 style="color:#01308e;">Metric to be used:</h3>

**F2 Score : ((1 + (2)^2) * Precision * Recall) / (((2)^2 * Precision) + Recall)**

<h3 style="color:#01308e;">Datasets:</h3>

**<u>3 datasets :</u><br><br>
1)Median imputed Median dataset.<br>
2)Median imputed Adasyn dataset.<br>
3)Median imputed Smotetomek dataset.<br><br>
I will feed these 3 datasets into MultiLayered Perceptron.**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import fbeta_score

In [3]:
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

In [4]:
import datetime
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.python.keras.utils import tf_utils
from tensorflow.keras.models import Model,Sequential
from tensorflow.python.keras.utils.io_utils import path_to_string

In [5]:
import os
os.cpu_count()

8

**Get the data**

In [6]:
new_standard_train_median_all_features_df=pd.read_pickle("new_standard_train_median_all_features_df.pickle")
y_train_median_all_features=pd.read_pickle("y_train_median_all_features.pickle")
new_standard_test_median_all_features_df=pd.read_pickle("new_standard_test_median_all_features_df.pickle")
y_test_median_all_features=pd.read_pickle("y_test_median_all_features.pickle")

new_x_adasyn_df=pd.read_pickle("new_x_adasyn_df.pickle")
y_adasyn=pd.read_pickle("y_adasyn.pickle")
new_standard_test_adasyn_all_features_df=pd.read_pickle("new_standard_test_adasyn_all_features_df.pickle")
y_test_adasyn_all_features=pd.read_pickle("y_test_adasyn_all_features.pickle")

new_x_smotetomek_df=pd.read_pickle("new_x_smotetomek_df.pickle")
y_smotetomek=pd.read_pickle("y_smotetomek.pickle")
new_standard_test_smotetomek_all_features_df=pd.read_pickle("new_standard_test_smotetomek_all_features_df.pickle")
y_test_smotetomek_all_features=pd.read_pickle("y_test_smotetomek_all_features.pickle")

**I am taking 20% of the data as cv and the rest I will use it as train data.**

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(
    new_standard_train_median_all_features_df, y_train_median_all_features,
    test_size=0.20, stratify=y_train_median_all_features)

In [8]:
input_dim=new_standard_train_median_all_features_df.shape[1]
input_dim

127

In [9]:
#We need to convert our pandas data into numpy before giving it to Tensorflow.
x_train=x_train.to_numpy()
x_cv=x_cv.to_numpy()

In [10]:
#pd.get_dummies() converts a list of categorical variables into one hot encoded variables.
#We need our categorical variables to be in the form of one hot encoded variables.
y_train = pd.get_dummies(y_train).values
y_cv = pd.get_dummies(y_cv).values

In [11]:
print("type(x_train) : ",type(x_train))
print("type(y_train) : ",type(y_train))
print("type(x_cv) : ",type(x_cv))
print("type(y_cv) : ",type(y_cv))
print("x_train.shape : ",x_train.shape)
print("y_train.shape : ",y_train.shape)
print("x_cv.shape : ",x_cv.shape)
print("y_cv.shape : ",y_cv.shape)

type(x_train) :  <class 'numpy.ndarray'>
type(y_train) :  <class 'numpy.ndarray'>
type(x_cv) :  <class 'numpy.ndarray'>
type(y_cv) :  <class 'numpy.ndarray'>
x_train.shape :  (38400, 127)
y_train.shape :  (38400, 2)
x_cv.shape :  (9600, 127)
y_cv.shape :  (9600, 2)


### MLP

In [7]:
def fit_the_model(x_train,y_train,x_cv,y_cv,input_dim,epochs,batch_size,data_types,unit1,dropout1,unit2,dropout2,unit3,dropout3,unit4,dropout4,unit5,dropout5,rate,act=None):

    tf.keras.backend.clear_session()

    try:
        del model
        del Model
        print("Old Model deleted.")
    except(NameError):
        print("No Model yet.")
        pass

    tf.keras.backend.clear_session()     
    
    #For tensorflow.keras dropout rate, is a value between 0 and 1. Fraction of the input units to drop. 
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(unit1, input_shape=(input_dim,),activation=act),
        tf.keras.layers.Dropout(dropout1),
        tf.keras.layers.Dense(unit2,activation=act),
        tf.keras.layers.Dropout(dropout2),
        tf.keras.layers.Dense(unit3,activation=act),
        tf.keras.layers.Dropout(dropout3),
        tf.keras.layers.Dense(unit4,activation=act),
        tf.keras.layers.Dropout(dropout4),
        tf.keras.layers.Dense(unit5,activation=act),
        tf.keras.layers.Dropout(dropout5),
        tf.keras.layers.Dense(2,activation='softmax')
    ])
    model.summary()

    
    #Setting a callback to save the best model weight value. 
    checkpoint_filepath = str(data_types)+"_checkpoint_"+str(unit1)+"_"\
        +str(dropout1)+"_"\
        +str(unit2)+"_"\
        +str(dropout2)+"_"\
        +str(unit3)+"_"\
        +str(dropout3)+"_"\
        +str(unit4)+"_"\
        +str(dropout4)+"_"\
        +str(unit5)+"_"\
        +str(dropout5)+"_"+str(act)+"_"+str(rate)+"/checkpoint_"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    
    #Compiling the model.
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=rate), loss='categorical_crossentropy')
    
    #Training the model.
    model.fit(x_train,y_train,epochs=40,verbose=0,batch_size=batch_size,
              callbacks=[custom_Metrics(validation_data=(x_cv, y_cv),
                                        filepath=checkpoint_filepath,
                                        save_freq='epoch')])
    return model

In [8]:
#This custom_metrics_class will inherit the methods of the Base class Callback ==> Inheritance 
#class DerivedClassName(modname.BaseClassName)
#class DerivedClassName(BaseClassName)
class custom_Metrics(tf.keras.callbacks.ModelCheckpoint,tf.keras.callbacks.Callback):
    def __init__(self,validation_data,**kwargs): 
        
        super(custom_Metrics, self).__init__(**kwargs)  
        
        self.validation_data=validation_data
        print("type(validation_data)",type(self.validation_data))
        print('validation_data[0]', self.validation_data[0].shape)
        print('validation_data[1]', self.validation_data[1].shape)
        
    def on_epoch_begin(self, epoch, logs=None):
        self._current_epoch = epoch
     
    def on_epoch_end(self, epoch, logs):
        val_targ = self.validation_data[1]   
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
    
        f2_score=np.round(fbeta_score(val_targ[:,1], val_predict[:,1], beta=2,average="binary", pos_label=1),6)

        print(f'Epoch : {epoch} \t\t\t val_f2: {f2_score}')
    
        self.epochs_since_last_save += 1
        if self.save_freq == 'epoch':
            self.save_model(epoch=epoch, logs=logs)
    
    def save_model(self, epoch, logs={}):
        """This is a method used for saving the model, this method has been customised accordingly.
        (original function: def _save_model in tf.keras.callbacks.ModelCheckpoint)
        """

        logs = tf_utils.to_numpy_or_python_type(logs)
        
        filepath = self._get_file_path(epoch, logs)+"_Epoch_%03d"% (epoch)

        print('Epoch %03d: saving weights only to %s\n\n' % (epoch, filepath))
        self.model.save_weights(filepath, overwrite=True, options=self._options)

**Trying with median based data.**

In [14]:
epochs = 40
batch_size=1000
activations=[None,"relu","selu","elu"]
rates=[0.0001,0.001,0.01]

In [15]:
# Configuration of number of hidden units in a layer followed by the dropout rate for each layer.
l=[
[100,.5,50,.4,20,.3,10,.2,5,.1],
[200,.8,100,.4,50,.6,20,.2,5,.5],
[400,.8,200,.4,100,.6,50,.2,10,.5],
[400,.4,200,.6,100,.8,50,.4,10,.2],
[50,.5,40,.4,30,.3,20,.2,10,.1],
[200,.1,300,.2,400,.3,500,.4,600,.5]
]

In [ ]:
%%time
for config in l:
    for act in activations:
        for rate in rates:
            unit1=config[0]
            dropout1=config[1]
            unit2=config[2]
            dropout2=config[3]
            unit3=config[4]
            dropout3=config[5]
            unit4=config[6]
            dropout4=config[7]
            unit5=config[8]
            dropout5=config[9]

            model = fit_the_model(x_train,y_train,x_cv,y_cv,input_dim,epochs,
                                  batch_size,"median_data",
                                  unit1,dropout1,unit2,dropout2,unit3,dropout3,unit4,dropout4,unit5,dropout5,rate,act)

No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               12800     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)   

Epoch : 36 			 val_f2: 0.747487
Epoch 036: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201111-232926_Epoch_036


Epoch : 37 			 val_f2: 0.737705
Epoch 037: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201111-232926_Epoch_037


Epoch : 38 			 val_f2: 0.743073
Epoch 038: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201111-232926_Epoch_038


Epoch : 39 			 val_f2: 0.728771
Epoch 039: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201111-232926_Epoch_039


No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               12800     
_________________________________________________________________
dr

Epoch : 33 			 val_f2: 0.741935
Epoch 033: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201111-232950_Epoch_033


Epoch : 34 			 val_f2: 0.741935
Epoch 034: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201111-232950_Epoch_034


Epoch : 35 			 val_f2: 0.783439
Epoch 035: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201111-232950_Epoch_035


Epoch : 36 			 val_f2: 0.799492
Epoch 036: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201111-232950_Epoch_036


Epoch : 37 			 val_f2: 0.696452
Epoch 037: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201111-232950_Epoch_037


Epoch : 38 			 val_f2: 0.707733
Epoch 038: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0

Epoch : 30 			 val_f2: 0.66
Epoch 030: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201111-233014_Epoch_030


Epoch : 31 			 val_f2: 0.502793
Epoch 031: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201111-233014_Epoch_031


Epoch : 32 			 val_f2: 0.735677
Epoch 032: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201111-233014_Epoch_032


Epoch : 33 			 val_f2: 0.670651
Epoch 033: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201111-233014_Epoch_033


Epoch : 34 			 val_f2: 0.820988
Epoch 034: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201111-233014_Epoch_034


Epoch : 35 			 val_f2: 0.707733
Epoch 035: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/check

Epoch : 27 			 val_f2: 0.091324
Epoch 027: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201111-233039_Epoch_027


Epoch : 28 			 val_f2: 0.068913
Epoch 028: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201111-233039_Epoch_028


Epoch : 29 			 val_f2: 0.069018
Epoch 029: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201111-233039_Epoch_029


Epoch : 30 			 val_f2: 0.053929
Epoch 030: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201111-233039_Epoch_030


Epoch : 31 			 val_f2: 0.031008
Epoch 031: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201111-233039_Epoch_031


Epoch : 32 			 val_f2: 0.03876
Epoch 032: saving weights only to median_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_re

...

Epoch : 0 			 val_f2: 0.535957
Epoch 000: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-001705_Epoch_000


Epoch : 1 			 val_f2: 0.640854
Epoch 001: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-001705_Epoch_001


Epoch : 2 			 val_f2: 0.738126
Epoch 002: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-001705_Epoch_002


Epoch : 3 			 val_f2: 0.710526
Epoch 003: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-001705_Epoch_003


Epoch : 4 			 val_f2: 0.637584
Epoch 004: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-001705_Epoch_004


Epoch : 5 			 val_f2: 0.795892
Epoch 005: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.

Epoch : 0 			 val_f2: 0.678733
Epoch 000: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-001908_Epoch_000


Epoch : 1 			 val_f2: 0.578313
Epoch 001: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-001908_Epoch_001


Epoch : 2 			 val_f2: 0.726121
Epoch 002: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-001908_Epoch_002


Epoch : 3 			 val_f2: 0.72318
Epoch 003: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-001908_Epoch_003


Epoch : 4 			 val_f2: 0.737898
Epoch 004: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-001908_Epoch_004


Epoch : 5 			 val_f2: 0.0078
Epoch 005: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.

type(validation_data) <class 'tuple'>
validation_data[0] (9600, 127)
validation_data[1] (9600, 2)
Epoch : 0 			 val_f2: 0.273312
Epoch 000: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-002114_Epoch_000


Epoch : 1 			 val_f2: 0.47014
Epoch 001: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-002114_Epoch_001


Epoch : 2 			 val_f2: 0.689441
Epoch 002: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-002114_Epoch_002


Epoch : 3 			 val_f2: 0.726928
Epoch 003: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-002114_Epoch_003


Epoch : 4 			 val_f2: 0.742386
Epoch 004: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-002114_Epoch_004


Epoch : 5 			

Epoch : 0 			 val_f2: 0.789474
Epoch 000: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-002314_Epoch_000


Epoch : 1 			 val_f2: 0.812121
Epoch 001: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-002314_Epoch_001


Epoch : 2 			 val_f2: 0.780457
Epoch 002: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-002314_Epoch_002


Epoch : 3 			 val_f2: 0.77474
Epoch 003: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-002314_Epoch_003


Epoch : 4 			 val_f2: 0.806658
Epoch 004: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-002314_Epoch_004


Epoch : 5 			 val_f2: 0.81749
Epoch 005: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0

Epoch : 0 			 val_f2: 0.599078
Epoch 000: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-002514_Epoch_000


Epoch : 1 			 val_f2: 0.550062
Epoch 001: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-002514_Epoch_001


Epoch : 2 			 val_f2: 0.38036
Epoch 002: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-002514_Epoch_002


Epoch : 3 			 val_f2: 0.530201
Epoch 003: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-002514_Epoch_003


Epoch : 4 			 val_f2: 0.629096
Epoch 004: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-002514_Epoch_004


Epoch : 5 			 val_f2: 0.794504
Epoch 005: saving weights only to median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_6

**Trying with adasyn based data.**

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(
    new_x_adasyn_df, y_adasyn,
    test_size=0.20, stratify=y_adasyn)

In [13]:
input_dim=new_x_adasyn_df.shape[1]
input_dim

144

In [14]:
#We need to convert our pandas data into numpy before giving it to Tensorflow.
x_train=x_train.to_numpy()
x_cv=x_cv.to_numpy()

In [15]:
#pd.get_dummies() converts a list of categorical variables into one hot encoded variables.
#We need our categorical variables to be in the form of one hot encoded variables.
y_train = pd.get_dummies(y_train).values
y_cv = pd.get_dummies(y_cv).values

In [16]:
print("type(x_train) : ",type(x_train))
print("type(y_train) : ",type(y_train))
print("type(x_cv) : ",type(x_cv))
print("type(y_cv) : ",type(y_cv))
print("x_train.shape : ",x_train.shape)
print("y_train.shape : ",y_train.shape)
print("x_cv.shape : ",x_cv.shape)
print("y_cv.shape : ",y_cv.shape)

type(x_train) :  <class 'numpy.ndarray'>
type(y_train) :  <class 'numpy.ndarray'>
type(x_cv) :  <class 'numpy.ndarray'>
type(y_cv) :  <class 'numpy.ndarray'>
x_train.shape :  (66074, 144)
y_train.shape :  (66074, 2)
x_cv.shape :  (16519, 144)
y_cv.shape :  (16519, 2)


In [ ]:
%%time
for config in l:
    for act in activations:
        for rate in rates:
            unit1=config[0]
            dropout1=config[1]
            unit2=config[2]
            dropout2=config[3]
            unit3=config[4]
            dropout3=config[5]
            unit4=config[6]
            dropout4=config[7]
            unit5=config[8]
            dropout5=config[9]

            model = fit_the_model(x_train,y_train,x_cv,y_cv,input_dim,epochs,
                                  batch_size,"adasyn_data",
                                  unit1,dropout1,unit2,dropout2,unit3,dropout3,unit4,dropout4,unit5,dropout5,rate,act)

No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               14500     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)   

Epoch : 36 			 val_f2: 0.93647
Epoch 036: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-005716_Epoch_036


Epoch : 37 			 val_f2: 0.937109
Epoch 037: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-005716_Epoch_037


Epoch : 38 			 val_f2: 0.937207
Epoch 038: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-005716_Epoch_038


Epoch : 39 			 val_f2: 0.936887
Epoch 039: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-005716_Epoch_039


No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               14500     
_________________________________________________________________
dro

Epoch : 33 			 val_f2: 0.941419
Epoch 033: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-005755_Epoch_033


Epoch : 34 			 val_f2: 0.93596
Epoch 034: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-005755_Epoch_034


Epoch : 35 			 val_f2: 0.941753
Epoch 035: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-005755_Epoch_035


Epoch : 36 			 val_f2: 0.940291
Epoch 036: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-005755_Epoch_036


Epoch : 37 			 val_f2: 0.936895
Epoch 037: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-005755_Epoch_037


Epoch : 38 			 val_f2: 0.942587
Epoch 038: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.

Epoch : 30 			 val_f2: 0.764695
Epoch 030: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-005834_Epoch_030


Epoch : 31 			 val_f2: 0.931178
Epoch 031: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-005834_Epoch_031


Epoch : 32 			 val_f2: 0.936797
Epoch 032: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-005834_Epoch_032


Epoch : 33 			 val_f2: 0.940636
Epoch 033: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-005834_Epoch_033


Epoch : 34 			 val_f2: 0.936112
Epoch 034: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-005834_Epoch_034


Epoch : 35 			 val_f2: 0.901205
Epoch 035: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/c

Epoch : 27 			 val_f2: 0.962443
Epoch 027: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-005914_Epoch_027


Epoch : 28 			 val_f2: 0.963629
Epoch 028: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-005914_Epoch_028


Epoch : 29 			 val_f2: 0.964546
Epoch 029: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-005914_Epoch_029


Epoch : 30 			 val_f2: 0.966137
Epoch 030: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-005914_Epoch_030


Epoch : 31 			 val_f2: 0.96734
Epoch 031: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-005914_Epoch_031


Epoch : 32 			 val_f2: 0.967701
Epoch 032: saving weights only to adasyn_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_re

...

Epoch : 0 			 val_f2: 0.966473
Epoch 000: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-021750_Epoch_000


Epoch : 1 			 val_f2: 0.981718
Epoch 001: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-021750_Epoch_001


Epoch : 2 			 val_f2: 0.98767
Epoch 002: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-021750_Epoch_002


Epoch : 3 			 val_f2: 0.994707
Epoch 003: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-021750_Epoch_003


Epoch : 4 			 val_f2: 0.994673
Epoch 004: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-021750_Epoch_004


Epoch : 5 			 val_f2: 0.997689
Epoch 005: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3

Epoch : 0 			 val_f2: 0.957119
Epoch 000: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-022124_Epoch_000


Epoch : 1 			 val_f2: 0.966644
Epoch 001: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-022124_Epoch_001


Epoch : 2 			 val_f2: 0.973455
Epoch 002: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-022124_Epoch_002


Epoch : 3 			 val_f2: 0.979247
Epoch 003: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-022124_Epoch_003


Epoch : 4 			 val_f2: 0.981935
Epoch 004: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-022124_Epoch_004


Epoch : 5 			 val_f2: 0.985434
Epoch 005: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500

type(validation_data) <class 'tuple'>
validation_data[0] (16519, 144)
validation_data[1] (16519, 2)
Epoch : 0 			 val_f2: 0.945867
Epoch 000: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-022443_Epoch_000


Epoch : 1 			 val_f2: 0.953895
Epoch 001: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-022443_Epoch_001


Epoch : 2 			 val_f2: 0.962053
Epoch 002: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-022443_Epoch_002


Epoch : 3 			 val_f2: 0.965273
Epoch 003: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-022443_Epoch_003


Epoch : 4 			 val_f2: 0.971313
Epoch 004: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-022443_Epoch_004


Epoch : 5 

Epoch : 0 			 val_f2: 0.971858
Epoch 000: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-022813_Epoch_000


Epoch : 1 			 val_f2: 0.985975
Epoch 001: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-022813_Epoch_001


Epoch : 2 			 val_f2: 0.987576
Epoch 002: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-022813_Epoch_002


Epoch : 3 			 val_f2: 0.993201
Epoch 003: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-022813_Epoch_003


Epoch : 4 			 val_f2: 0.997913
Epoch 004: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-022813_Epoch_004


Epoch : 5 			 val_f2: 0.997714
Epoch 005: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500

type(validation_data) <class 'tuple'>
validation_data[0] (16519, 144)
validation_data[1] (16519, 2)
Epoch : 0 			 val_f2: 0.95787
Epoch 000: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-023142_Epoch_000


Epoch : 1 			 val_f2: 0.970739
Epoch 001: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-023142_Epoch_001


Epoch : 2 			 val_f2: 0.980791
Epoch 002: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-023142_Epoch_002


Epoch : 3 			 val_f2: 0.98551
Epoch 003: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-023142_Epoch_003


Epoch : 4 			 val_f2: 0.99298
Epoch 004: saving weights only to adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-023142_Epoch_004


Epoch : 5 			 val_f2: 0

**Trying with smotetomek based data.**

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(
    new_x_smotetomek_df, y_smotetomek,
    test_size=0.20, stratify=y_smotetomek)

In [10]:
input_dim=new_x_smotetomek_df.shape[1]
input_dim

117

In [11]:
#We need to convert our pandas data into numpy before giving it to Tensorflow.
x_train=x_train.to_numpy()
x_cv=x_cv.to_numpy()

In [12]:
#pd.get_dummies() converts a list of categorical variables into one hot encoded variables.
#We need our categorical variables to be in the form of one hot encoded variables.
y_train = pd.get_dummies(y_train).values
y_cv = pd.get_dummies(y_cv).values

In [13]:
print("type(x_train) : ",type(x_train))
print("type(y_train) : ",type(y_train))
print("type(x_cv) : ",type(x_cv))
print("type(y_cv) : ",type(y_cv))
print("x_train.shape : ",x_train.shape)
print("y_train.shape : ",y_train.shape)
print("x_cv.shape : ",x_cv.shape)
print("y_cv.shape : ",y_cv.shape)

type(x_train) :  <class 'numpy.ndarray'>
type(y_train) :  <class 'numpy.ndarray'>
type(x_cv) :  <class 'numpy.ndarray'>
type(y_cv) :  <class 'numpy.ndarray'>
x_train.shape :  (66078, 117)
y_train.shape :  (66078, 2)
x_cv.shape :  (16520, 117)
y_cv.shape :  (16520, 2)


In [ ]:
%%time
for config in l:
    for act in activations:
        for rate in rates:
            unit1=config[0]
            dropout1=config[1]
            unit2=config[2]
            dropout2=config[3]
            unit3=config[4]
            dropout3=config[5]
            unit4=config[6]
            dropout4=config[7]
            unit5=config[8]
            dropout5=config[9]

            model = fit_the_model(x_train,y_train,x_cv,y_cv,input_dim,epochs,
                                  batch_size,"smotetomek_data",
                                  unit1,dropout1,unit2,dropout2,unit3,dropout3,unit4,dropout4,unit5,dropout5,rate,act)

No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               11800     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)   

Epoch : 36 			 val_f2: 0.963763
Epoch 036: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-031547_Epoch_036


Epoch : 37 			 val_f2: 0.963405
Epoch 037: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-031547_Epoch_037


Epoch : 38 			 val_f2: 0.964905
Epoch 038: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-031547_Epoch_038


Epoch : 39 			 val_f2: 0.963286
Epoch 039: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.0001/checkpoint_20201112-031547_Epoch_039


No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               11800     
____________________________________________________

Epoch : 32 			 val_f2: 0.963211
Epoch 032: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-031624_Epoch_032


Epoch : 33 			 val_f2: 0.96271
Epoch 033: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-031624_Epoch_033


Epoch : 34 			 val_f2: 0.960955
Epoch 034: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-031624_Epoch_034


Epoch : 35 			 val_f2: 0.962073
Epoch 035: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-031624_Epoch_035


Epoch : 36 			 val_f2: 0.960835
Epoch 036: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.001/checkpoint_20201112-031624_Epoch_036


Epoch : 37 			 val_f2: 0.962797
Epoch 037: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_

Epoch : 28 			 val_f2: 0.964035
Epoch 028: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-031700_Epoch_028


Epoch : 29 			 val_f2: 0.963386
Epoch 029: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-031700_Epoch_029


Epoch : 30 			 val_f2: 0.965158
Epoch 030: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-031700_Epoch_030


Epoch : 31 			 val_f2: 0.936579
Epoch 031: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-031700_Epoch_031


Epoch : 32 			 val_f2: 0.947664
Epoch 032: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_None_0.01/checkpoint_20201112-031700_Epoch_032


Epoch : 33 			 val_f2: 0.958142
Epoch 033: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_

Epoch : 24 			 val_f2: 0.97064
Epoch 024: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-031735_Epoch_024


Epoch : 25 			 val_f2: 0.971128
Epoch 025: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-031735_Epoch_025


Epoch : 26 			 val_f2: 0.97121
Epoch 026: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-031735_Epoch_026


Epoch : 27 			 val_f2: 0.971503
Epoch 027: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-031735_Epoch_027


Epoch : 28 			 val_f2: 0.971956
Epoch 028: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4_20_0.3_10_0.2_5_0.1_relu_0.0001/checkpoint_20201112-031735_Epoch_028


Epoch : 29 			 val_f2: 0.972023
Epoch 029: saving weights only to smotetomek_data_checkpoint_100_0.5_50_0.4

...

Epoch : 0 			 val_f2: 0.979048
Epoch 000: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-042505_Epoch_000


Epoch : 1 			 val_f2: 0.984253
Epoch 001: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-042505_Epoch_001


Epoch : 2 			 val_f2: 0.986523
Epoch 002: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-042505_Epoch_002


Epoch : 3 			 val_f2: 0.989943
Epoch 003: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-042505_Epoch_003


Epoch : 4 			 val_f2: 0.994505
Epoch 004: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.001/checkpoint_20201112-042505_Epoch_004


Epoch : 5 			 val_f2: 0.995183
Epoch 005: saving weights only to smotetomek_data_checkpoin

Epoch : 0 			 val_f2: 0.945612
Epoch 000: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-042808_Epoch_000


Epoch : 1 			 val_f2: 0.974792
Epoch 001: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-042808_Epoch_001


Epoch : 2 			 val_f2: 0.979559
Epoch 002: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-042808_Epoch_002


Epoch : 3 			 val_f2: 0.984188
Epoch 003: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-042808_Epoch_003


Epoch : 4 			 val_f2: 0.984923
Epoch 004: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_selu_0.01/checkpoint_20201112-042808_Epoch_004


Epoch : 5 			 val_f2: 0.984904
Epoch 005: saving weights only to smotetomek_data_checkpoint_200

Epoch : 0 			 val_f2: 0.965965
Epoch 000: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-043111_Epoch_000


Epoch : 1 			 val_f2: 0.971037
Epoch 001: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-043111_Epoch_001


Epoch : 2 			 val_f2: 0.975613
Epoch 002: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-043111_Epoch_002


Epoch : 3 			 val_f2: 0.978046
Epoch 003: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-043111_Epoch_003


Epoch : 4 			 val_f2: 0.980644
Epoch 004: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.0001/checkpoint_20201112-043111_Epoch_004


Epoch : 5 			 val_f2: 0.981022
Epoch 005: saving weights only to smotetomek_data_checkpoin

Epoch : 0 			 val_f2: 0.982244
Epoch 000: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-043415_Epoch_000


Epoch : 1 			 val_f2: 0.987799
Epoch 001: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-043415_Epoch_001


Epoch : 2 			 val_f2: 0.989186
Epoch 002: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-043415_Epoch_002


Epoch : 3 			 val_f2: 0.991929
Epoch 003: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-043415_Epoch_003


Epoch : 4 			 val_f2: 0.993967
Epoch 004: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.001/checkpoint_20201112-043415_Epoch_004


Epoch : 5 			 val_f2: 0.99594
Epoch 005: saving weights only to smotetomek_data_checkpoint_200_

Epoch : 0 			 val_f2: 0.977131
Epoch 000: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-043719_Epoch_000


Epoch : 1 			 val_f2: 0.976011
Epoch 001: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-043719_Epoch_001


Epoch : 2 			 val_f2: 0.983706
Epoch 002: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-043719_Epoch_002


Epoch : 3 			 val_f2: 0.990153
Epoch 003: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-043719_Epoch_003


Epoch : 4 			 val_f2: 0.99202
Epoch 004: saving weights only to smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_elu_0.01/checkpoint_20201112-043719_Epoch_004


Epoch : 5 			 val_f2: 0.993198
Epoch 005: saving weights only to smotetomek_data_checkpoint_200_0.1_3

**Trying with the highest median data weights**

In [12]:
#median
np.sort([0.308511,0.379908,0.437428,0.496384,0.526499,0.562593,0.595813,0.635335,0.667921,0.684932,0.712058,0.726987,0.738699,0.741163,0.747126,0.75,0.734767,0.709046,0.720391,0.730676,0.707257,0.726496,0.730061,0.734568,0.715182,0.715182,0.726817,0.735661,0.7375,0.734003,0.728643,0.72327,0.72956,0.71066,0.72335,0.757196,0.747487,0.737705,0.743073,0.728771,0.654475,0.727924,0.681818,0.682111,0.775434,0.625823,0.811648,0.780457,0.728093,0.789141,0.799748,0.766284,0.765766,0.803797,0.828144,0.830235,0.718954,0.749354,0.71708,0.807062,0.819775,0.605653,0.794155,0.77707,0.830235,0.801527,0.82702,0.747423,0.749354,0.741222,0.750963,0.787802,0.785441,0.741935,0.741935,0.783439,0.799492,0.696452,0.707733,0.752896,0.486301,0.799523,0.616622,0.559345,0.611559,0.792383,0.771684,0.785176,0.688312,0.759097,0.753512,0.602981,0.69281,0.690789,0.731707,0.792453,0.798479,0.791139,0.786445,0.74026,0.807259,0.787154,0.650134,0.587432,0.750963,0.74026,0.799748,0.82716,0.818966,0.612382,0.66,0.502793,0.735677,0.670651,0.820988,0.707733,0.775641,0.824242,0.832321,0.787802,0.19186,0.123907,0.119581,0.105422,0.11976,0.133531,0.231884,0.191176,0.178042,0.164425,0.164671,0.142857,0.135542,0.142429,0.170877,0.163447,0.184911,0.170623,0.17037,0.163447,0.149031,0.156483,0.149254,0.142429,0.142429,0.106061,0.091463,0.091324,0.068913,0.069018,0.053929,0.031008,0.03876,0.03876,0.031056,0.015576,0.015576,0.023328,0.031056,0.015576,0.0,0.0,0.0,0.0,0.0078,0.015576,0.023328,0.015576,0.015576,0.076453,0.235988,0.542005,0.738854,0.753512,0.733766,0.788146,0.777497,0.751928,0.736434,0.787802,0.762821,0.797468,0.812343,0.767742,0.812343,0.813215,0.820707,0.822864,0.801527,0.811153,0.813215,0.825977,0.832282,0.84799,0.849057,0.840708,0.826972,0.848101,0.828025,0.844981,0.023328,0.251108,0.22997,0.632334,0.258112,0.754476,0.771684,0.770701,0.546335,0.772201,0.509066,0.637584,0.688742,0.736636,0.803342,0.795892,0.808729,0.815148,0.801282,0.810983,0.784695,0.838628,0.792526,0.771725,0.803342,0.836502,0.788114,0.816195,0.833333,0.815006,0.75,0.776762,0.832266,0.804376,0.774278,0.78776,0.843909,0.842912,0.79118,0.844981,0.313825,0.339982,0.365196,0.430832,0.443548,0.454248,0.469707,0.479599,0.476735,0.45679,0.416294,0.397129,0.372285,0.34292,0.318656,0.313631,0.312891,0.29525,0.286086,0.275229,0.271164,0.260695,0.269542,0.264586,0.284168,0.304878,0.306958,0.294925,0.283541,0.297784,0.32967,0.330124,0.32967,0.330124,0.336077,0.363014,0.394558,0.418919,0.431849,0.431267,0.188008,0.36452,0.554217,0.66787,0.707434,0.764636,0.786982,0.795053,0.804529,0.820452,0.812721,0.824437,0.835294,0.832349,0.834341,0.843301,0.833333,0.845886,0.83632,0.837321,0.832321,0.848057,0.837349,0.845499,0.853365,0.856974,0.838396,0.815951,0.835366,0.862485,0.844471,0.868263,0.844471,0.828221,0.857488,0.863747,0.848708,0.838471,0.846626,0.848596,0.015528,0.106383,0.360679,0.710692,0.802326,0.738916,0.813539,0.778739,0.767836,0.818182,0.79878,0.787208,0.800493,0.794335,0.814677,0.785627,0.840544,0.803238,0.794743,0.82404,0.853659,0.851449,0.819775,0.845771,0.782443,0.87037,0.872525,0.880579,0.85199,0.76129,0.772201,0.840438,0.849876,0.8375,0.874083,0.85199,0.851852,0.868227,0.875462,0.816327,0.295305,0.331405,0.357143,0.386518,0.415293,0.440666,0.454545,0.461184,0.454545,0.470738,0.472122,0.485661,0.494297,0.482412,0.496926,0.505735,0.521053,0.538213,0.535117,0.54732,0.551724,0.579793,0.561404,0.574645,0.573964,0.580838,0.584337,0.584337,0.587879,0.625,0.618182,0.605134,0.607362,0.625,0.629584,0.640244,0.644172,0.628853,0.640394,0.648148,0.103164,0.052006,0.045662,0.045662,0.083207,0.090772,0.098336,0.128205,0.135747,0.150376,0.221566,0.30479,0.223214,0.243722,0.285505,0.366379,0.448179,0.345324,0.542582,0.487465,0.493741,0.524862,0.609756,0.591837,0.591837,0.550964,0.62753,0.6393,0.592643,0.707672,0.675134,0.680908,0.656836,0.745098,0.764249,0.762274,0.763708,0.768229,0.793548,0.719947,0.0,0.0,0.0,0.0,0.0,0.779302,0.779537,0.641534,0.699482,0.788509,0.794045,0.844391,0.525588,0.747724,0.837379,0.798005,0.765766,0.797194,0.823899,0.797194,0.789809,0.766234,0.841646,0.867225,0.785441,0.829146,0.860149,0.85335,0.857503,0.851334,0.858396,0.82702,0.792526,0.77474,0.818299,0.838628,0.795306,0.827985,0.855181,0.819355,0.273423,0.3563,0.42033,0.465116,0.499671,0.540925,0.561497,0.589983,0.607176,0.613527,0.613161,0.616292,0.615819,0.622837,0.630153,0.62724,0.609756,0.594796,0.588972,0.593434,0.597964,0.605096,0.612245,0.599227,0.568627,0.551905,0.541612,0.540184,0.523179,0.518617,0.558476,0.547493,0.55409,0.560686,0.560686,0.560686,0.570292,0.60705,0.606258,0.601307,0.63148,0.607311,0.556274,0.559168,0.628205,0.698992,0.734003,0.7375,0.721519,0.720663,0.752212,0.710594,0.742638,0.750636,0.740026,0.757381,0.732815,0.781449,0.779468,0.770218,0.776081,0.772669,0.78481,0.783818,0.759331,0.790139,0.76972,0.780457,0.759331,0.714286,0.795165,0.793147,0.690789,0.783439,0.807062,0.757381,0.771208,0.788804,0.770218,0.802781,0.618687,0.578947,0.709677,0.781841,0.717982,0.780856,0.714286,0.693717,0.740979,0.741935,0.628342,0.629183,0.559345,0.730769,0.519391,0.704961,0.747423,0.544828,0.393983,0.658245,0.72549,0.70118,0.657371,0.712418,0.774112,0.621658,0.731865,0.731707,0.616622,0.635877,0.740979,0.741935,0.664894,0.720779,0.71708,0.760309,0.723598,0.675497,0.669761,0.752896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031056,0.054096,0.084485,0.15861,0.099541,0.121951,0.18797,0.237389,0.251108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.282711,0.335599,0.361556,0.382752,0.40617,0.42246,0.441094,0.453242,0.46754,0.4813,0.492472,0.49904,0.510471,0.517928,0.523295,0.534483,0.54386,0.550817,0.555556,0.557955,0.565693,0.577778,0.585146,0.586677,0.589368,0.588006,0.595426,0.600159,0.607401,0.609362,0.615824,0.622937,0.62812,0.632858,0.640373,0.646404,0.645995,0.648228,0.647263,0.646288,0.474777,0.549683,0.072202,0.014368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007788,0.007776,0.007776,0.0,0.0,0.007776,0.007716,0.007704,0.00774,0.0,0.0,0.030628,0.80117,0.555556,0.736776,0.704315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.179818,0.239617,0.264901,0.281636,0.291399,0.30214,0.325592,0.387766,0.410959,0.428316,0.442804,0.454259,0.458922,0.472313,0.480132,0.490921,0.499658,0.507297,0.51264,0.517364,0.523673,0.530909,0.537555,0.544848,0.551827,0.561031,0.570444,0.569423,0.579937,0.584057,0.585907,0.592949,0.604708,0.604938,0.610465,0.617128,0.626058,0.640585,0.652742,0.65331,0.346715,0.503513,0.589448,0.610919,0.661425,0.713536,0.716544,0.651408,0.720046,0.686391,0.595533,0.429326,0.414474,0.348361,0.250716,0.317797,0.335196,0.298295,0.325779,0.199115,0.22694,0.211679,0.149254,0.112782,0.134328,0.112444,0.075415,0.075529,0.067771,0.030303,0.015291,0.007669,0.0,0.022936,0.140741,0.223988,0.430464,0.474308,0.621039,0.739183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.363075,0.46754,0.540634,0.592006,0.649406,0.675918,0.693642,0.684713,0.681063,0.68525,0.718492,0.658385,0.630517,0.624204,0.629771,0.616175,0.612113,0.634615,0.64578,0.612903,0.590143,0.602332,0.608021,0.647436,0.624196,0.604681,0.620957,0.616883,0.664962,0.648267,0.711563,0.662806,0.640362,0.679487,0.692308,0.648508,0.688546,0.727041,0.694981,0.694981,0.642857,0.588235,0.670038,0.735661,0.747487,0.662338,0.676203,0.776515,0.720721,0.742894,0.75641,0.793147,0.765766,0.817728,0.825062,0.778061,0.771684,0.781449,0.768246,0.810302,0.78105,0.791139,0.800757,0.788804,0.743855,0.764781,0.814394,0.811153,0.74026,0.814394,0.805838,0.819876,0.70118,0.804816,0.78105,0.803797,0.798212,0.772201,0.799492,0.782443,0.72695,0.770631,0.616798,0.677419,0.765823,0.777497,0.803797,0.794451,0.672823,0.798267,0.652463,0.685112,0.755814,0.7021,0.726329,0.720721,0.683311,0.58663,0.568493,0.713359,0.580601,0.481172,0.835396,0.469846,0.591837,0.809465,0.476858,0.780457,0.61828,0.747423,0.816708,0.657371,0.794451,0.773657,0.741935,0.72549,0.620783,0.824437,0.712418,0.733766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015552,0.007788,0.0078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0078,0.0,0.0,0.0,0.015576,0.015576,0.015576,0.023328,0.023328,0.054096,0.092025,0.107034,0.084485,0.121951,0.277372,0.448808,0.34632,0.430183,0.480501,0.493056,0.487465,0.40483,0.553279,0.564626,0.469188,0.554036,0.554036,0.572207,0.571429,0.572988,0.544077,0.544828,0.626685,0.519391,0.609756,0.526316,0.615697,0.655957,0.651007,0.691489,0.645161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.306005,0.409583,0.463127,0.491803,0.514608,0.518081,0.522599,0.518657,0.473396,0.471774,0.424809,0.39626,0.3463,0.328358,0.325413,0.300218,0.288351,0.278436,0.269762,0.269923,0.266234,0.257596,0.233645,0.22942,0.225718,0.186722,0.17337,0.154278,0.141243,0.134943,0.10745,0.09366,0.093795,0.093795,0.086831,0.079826,0.072886,0.05848,0.058565,0.065886,0.505181,0.583913,0.669399,0.707364,0.007485,0.0,0.0,0.0,0.03811,0.155556,0.06079,0.344101,0.326705,0.575163,0.539281,0.417246,0.724095,0.786982,0.680359,0.813539,0.760198,0.810651,0.806452,0.801205,0.805489,0.841924,0.804878,0.804734,0.811611,0.807601,0.819672,0.817757,0.816448,0.836237,0.844005,0.831368,0.835267,0.813953,0.838288,0.843373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.278689,0.342573,0.38175,0.393754,0.388769,0.405918,0.407759,0.404841,0.411664,0.403863,0.391621,0.347155,0.281186,0.267953,0.253025,0.245995,0.237248,0.191122,0.172414,0.156454,0.132275,0.116438,0.103734,0.091292,0.078236,0.071531,0.064841,0.05772,0.050872,0.051095,0.043988,0.036928,0.029674,0.014881,0.014948,0.022422,0.022455,0.022489,0.015038,0.015038,0.336538,0.078844,0.007474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015267,0.037879,0.202899,0.429741,0.312945,0.52027,0.410292,0.676203,0.755076,0.737913,0.625823,0.747487,0.795455,0.780098,0.790441,0.803991,0.791411,0.813397,0.80148,0.827295,0.830325,0.818291,0.822329,0.838359,0.821078,0.826193,0.835366,0.824108,0.829268,0.822168,0.70296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384428,0.492163,0.586026,0.650836,0.682904,0.706462,0.716743,0.692951,0.711722,0.674847,0.668317,0.621859,0.696517,0.678705,0.643312,0.63857,0.738916,0.716958,0.707134,0.71875,0.727612,0.724095,0.71608,0.721455,0.700637,0.702427,0.733249,0.740276,0.743073,0.728771,0.735108,0.725191,0.741445,0.728771,0.748731,0.743329,0.749682,0.750636,0.751592,0.75662,0.733634,0.74228,0.733831,0.768262,0.771644,0.772152,0.8,0.76087,0.79646,0.801768,0.806045,0.758929,0.782443,0.74026,0.781449,0.794155,0.801527,0.798479,0.786802,0.790816,0.753866,0.776081,0.707733,0.802781,0.821745,0.819775,0.797011,0.813517,0.752896,0.82397,0.785441,0.797468,0.816708,0.607287,0.817728,0.7638,0.775641,0.797468,0.801001,0.801768,0.410397,0.772613,0.798319,0.760456,0.771176,0.805882,0.777917,0.786164,0.810976,0.626667,0.743855,0.733766,0.815822,0.778481,0.709635,0.700389,0.813665,0.744544,0.785805,0.762821,0.587838,0.776942,0.739796,0.788486,0.761844,0.842044,0.827114,0.686016,0.771684,0.729032,0.802005,0.797739,0.64257,0.794743,0.828144,0.834306,0.77707,0.605442,0.795314,0.606469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0078,0.0078,0.031056,0.023328,0.031056,0.031056,0.023328,0.04644,0.061728,0.076805,0.084227,0.106707,0.091743,0.135952,0.143288,0.143072,0.186289,0.235641,0.283843,0.325145,0.0,0.0078,0.0078,0.015576,0.03876,0.069337,0.157186,0.432961,0.382436,0.554795,0.63172,0.714286,0.758929,0.704961,0.751928,0.806045,0.821745,0.822864,0.823899,0.827068,0.810983,0.807692,0.829114,0.84596,0.855346,0.855346,0.808729,0.820611,0.836502,0.841772,0.840652,0.831218,0.84799,0.85335,0.820513,0.865747,0.851334,0.846056,0.838668,0.871212,0.0,0.0,0.0,0.0,0.0,0.208955,0.158371,0.613333,0.645806,0.694087,0.568493,0.811486,0.372493,0.367435,0.556319,0.690571,0.230312,0.658602,0.673333,0.816583,0.798742,0.730263,0.677852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.379838,0.475194,0.522237,0.555556,0.594957,0.622937,0.644884,0.666075,0.689013,0.713597,0.715677,0.727912,0.718533,0.731584,0.732323,0.746421,0.7628,0.752577,0.763598,0.766002,0.77167,0.77167,0.776135,0.77577,0.777063,0.777897,0.778733,0.784314,0.786334,0.796791,0.795699,0.796791,0.800427,0.809061,0.812568,0.803859,0.812568,0.813319,0.812294,0.813449,0.661686,0.610212,0.569211,0.504711,0.497981,0.430556,0.380282,0.438719,0.530612,0.430791,0.460251,0.45582,0.399429,0.544077,0.392857,0.340087,0.461538,0.6917,0.585831,0.574555,0.648396,0.565395,0.770701,0.63172,0.748387,0.753866,0.789809,0.718954,0.828105,0.807062,0.745098,0.72644,0.768734,0.813367,0.842767,0.842839,0.797941,0.847029,0.848939,0.831218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.294003,0.335074,0.374396,0.413554,0.451106,0.475602,0.503927,0.514905,0.532503,0.543324,0.549569,0.567428,0.575916,0.587786,0.601093,0.605822,0.614035,0.61257,0.625,0.633803,0.64027,0.642434,0.665796,0.68487,0.686887,0.695574,0.70572,0.716292,0.725806,0.73487,0.737864,0.738916,0.742574,0.748231,0.766046,0.759754,0.755102,0.761759,0.768041,0.761421,0.259027,0.397351,0.385572,0.470738,0.694282,0.766629,0.773543,0.790725,0.804935,0.787879,0.822329,0.806643,0.815476,0.830368,0.843447,0.814095,0.847458,0.836364,0.832313,0.851716,0.836431,0.858896,0.847772,0.85625,0.846926,0.851198,0.863354,0.86272,0.842912,0.874384,0.845865,0.852273,0.842767,0.822622,0.851198,0.86185,0.85443,0.843909,0.857322,0.837596,0.0,0.0,0.615079,0.762195,0.812065,0.66273,0.251852,0.844391,0.619112,0.786164,0.614865,0.856454,0.842558,0.869565,0.876663,0.854241,0.877301,0.0,0.0,0.0,0.0,0.856481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304826,0.412286,0.463535,0.502717,0.53207,0.559846,0.592593,0.610288,0.61991,0.617925,0.642157,0.649949,0.664894,0.682068,0.696133,0.698864,0.695249,0.705053,0.702381,0.699761,0.684848,0.684597,0.669145,0.687732,0.671267,0.646607,0.680101,0.677215,0.667939,0.678074,0.641026,0.685279,0.653009,0.694268,0.678617,0.689655,0.696041,0.702427,0.681234,0.696041,0.647694,0.667461,0.677541,0.700758,0.602649,0.761965,0.736236,0.711514,0.733591,0.753512,0.740979,0.766284,0.771208,0.690789,0.800508,0.749354,0.790816,0.785441,0.794155,0.720779,0.770218,0.743855,0.787802,0.772669,0.800757,0.772669,0.779055,0.774648,0.752896,0.758355,0.766284,0.765306,0.771684,0.748387,0.772669,0.775641,0.723598,0.774648,0.765766,0.785805,0.564409,0.58311,0.74359,0.72445,0.735484,0.64,0.706806,0.751928,0.513176,0.717054,0.715215,0.651596,0.678851,0.66,0.79646,0.716146,0.619946,0.684211,0.824108,0.688742,0.729167,0.837408,0.711488,0.798742,0.717054,0.695538,0.817728,0.830235,0.785805,0.681217,0.810302,0.598639,0.544077,0.785805,0.793451,0.625841,0.815558,0.799492,0.837408,0.579809,0.067249,0.048611,0.015291,0.015504,0.015552,0.015552,0.023328,0.007788,0.015528,0.007788,0.007788,0.0,0.007776,0.015528,0.015528,0.023256,0.015528,0.03096,0.023256,0.023256,0.023256,0.023256,0.02322,0.007764,0.007776,0.007776,0.0,0.007776,0.015528,0.015528,0.02322,0.015504,0.015504,0.015504,0.015504,0.015504,0.007764,0.007764,0.007764,0.007764,0.03096,0.046225,0.0387,0.084485,0.107034,0.121766,0.158371,0.235641,0.369318,0.438108,0.463989,0.621693,0.605053,0.696931,0.688546,0.70232,0.753425,0.751263,0.748731,0.744275,0.779874,0.761421,0.786164,0.791457,0.817958,0.776515,0.784191,0.792453,0.80625,0.777497,0.808458,0.795739,0.782828,0.780457,0.795739,0.792453,0.803238,0.792453,0.801001,0.795455,0.0,0.0,0.0,0.015576,0.837264,0.799748,0.758355,0.678191,0.786802,0.706737,0.762274,0.836454,0.825183,0.846824,0.719947,0.830212,0.802548,0.838471,0.81749,0.810302,0.76524,0.800256,0.815287,0.859278,0.858396,0.826972,0.8375,0.835443,0.840708,0.838628,0.854115,0.816327,0.833333,0.855181,0.835459,0.86272,0.841837,0.825864,0.839695,0.869297,0.319911,0.380349,0.413512,0.435316,0.433526,0.427204,0.39884,0.366795,0.329218,0.291262,0.247432,0.193261,0.176349,0.136314,0.091848,0.065711,0.03736,0.032383,0.02681,0.020633,0.021038,0.021277,0.021708,0.022026,0.014815,0.014925,0.015015,0.015129,0.015152,0.015175,0.015221,0.015291,0.015361,0.015385,0.015408,0.015408,0.023112,0.023112,0.015432,0.015456,0.318386,0.112732,0.098592,0.108382,0.116448,0.15942,0.215827,0.314246,0.29703,0.361111,0.433604,0.542328,0.564304,0.570866,0.673443,0.648267,0.623377,0.69797,0.674807,0.685019,0.777363,0.76875,0.786241,0.770186,0.775,0.750636,0.799508,0.792079,0.799257,0.80148,0.809988,0.801242,0.808458,0.811648,0.807453,0.817844,0.826141,0.824108,0.804239,0.834363,0.239478,0.234949,0.624204,0.22997,0.562249,0.465925,0.726221,0.504841,0.79602,0.531768,0.776081,0.674603,0.805243,0.803797,0.820896,0.812183,0.847561,0.803571,0.844806,0.833333,0.803342,0.849754,0.66443,0.809465,0.845679,0.792526,0.819975,0.838548,0.833333,0.77575,0.853511,0.833333,0.771725,0.841646,0.814103,0.844893,0.818528,0.859473,0.836478,0.804376,0.212445,0.286732,0.328649,0.338937,0.321656,0.276326,0.244519,0.244131,0.231959,0.214758,0.199531,0.198317,0.185414,0.184713,0.175781,0.169935,0.171731,0.16,0.160858,0.142469,0.144429,0.145833,0.13986,0.141044,0.141643,0.135908,0.122478,0.122655,0.122832,0.130246,0.123547,0.123726,0.116618,0.109971,0.131965,0.118168,0.111276,0.119048,0.133531,0.147929,0.039062,0.0,0.0,0.0,0.0,0.0,0.0,0.015576,0.084098,0.143505,0.257732,0.40483,0.592992,0.696452,0.686016,0.745785,0.753247,0.797194,0.749674,0.782051,0.790488,0.771725,0.792839,0.799233,0.803797,0.802548,0.844806,0.81864,0.76524,0.814103,0.804598,0.809949,0.831218,0.856423,0.833333,0.823755,0.841772,0.854271,0.798969,0.823755,0.200297,0.669192,0.730769,0.756997,0.813665,0.797739,0.813367,0.800757,0.82702,0.832321,0.789141,0.789133,0.84375,0.820707,0.805627,0.879397,0.840764,0.787097,0.846824,0.82397,0.764249,0.828025,0.812903,0.809769,0.81306,0.765707,0.808538,0.812744,0.837596,0.841772,0.81306,0.826873,0.838668,0.84399,0.849298,0.809585,0.831202,0.733157,0.833333,0.813953,0.572312,0.684466,0.740093,0.765607,0.748762,0.719697,0.709719,0.73957,0.669701,0.721584,0.744949,0.769231,0.729834,0.779874,0.803238,0.761421,0.753165,0.79602,0.789141,0.712435,0.718912,0.739075,0.772669,0.772669,0.742638,0.785805,0.820802,0.791457,0.785805,0.794743,0.798479,0.808271,0.805031,0.753866,0.786802,0.785441,0.767263,0.780457,0.777635,0.798212,0.611183,0.699177,0.725095,0.800733,0.794903,0.775536,0.793651,0.730623,0.75188,0.707792,0.761421,0.765306,0.80402,0.812958,0.747126,0.772669,0.791771,0.771176,0.708763,0.74904,0.79878,0.781449,0.789801,0.741935,0.810302,0.781841,0.797468,0.74904,0.699208,0.735484,0.758355,0.757962,0.79646,0.809285,0.794335,0.798267,0.765306,0.723514,0.688073,0.757381,0.272418,0.724029,0.60567,0.802171,0.732484,0.795739,0.725191,0.667539,0.776942,0.686275,0.676741,0.622517,0.734536,0.751592,0.779874,0.752551,0.771684,0.83432,0.811138,0.783439,0.686016,0.739075,0.673712,0.584239,0.726329,0.7393,0.768742,0.790464,0.770701,0.548697,0.659121,0.789141,0.798267,0.73107,0.728218,0.711514,0.821918,0.734536,0.782228,0.766284,0.0,0.606654,0.665602,0.75575,0.769663,0.792254,0.79878,0.828402,0.81381,0.835351,0.835366,0.840438,0.836386,0.829298,0.820896,0.833333,0.842558,0.852341,0.852619,0.825,0.831266,0.836431,0.867971,0.858709,0.851852,0.849754,0.845679,0.83125,0.86983,0.838509,0.852905,0.84472,0.834379,0.861009,0.847772,0.84472,0.831234,0.848823,0.841709,0.858209,0.7314,0.798742,0.816456,0.818528,0.864662,0.835459,0.858396,0.818299,0.819355,0.863808,0.835459,0.822539,0.876543,0.830116,0.865992,0.880503,0.866834,0.840708,0.83014,0.868188,0.812744,0.873116,0.841772,0.844981,0.840708,0.855513,0.856599,0.84507,0.851334,0.858586,0.843909,0.834387,0.834395,0.85335,0.851472,0.868188,0.874525,0.910758,0.878635,0.879747,0.685897,0.258493,0.563187,0.793147,0.829177,0.832282,0.821832,0.841463,0.825921,0.762712,0.846626,0.803238,0.433855,0.674764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174,0.508475,0.468541,0.607843,0.649351,0.721584,0.715198,0.698577,0.746173,0.776081,0.807453,0.774648,0.780275,0.768742,0.788804,0.805243,0.801768,0.812183,0.806861,0.801527,0.764781,0.804816,0.805838,0.81749,0.816456,0.806861,0.817369,0.805627,0.810983,0.780645,0.806861,0.808729,0.817369,0.789133,0.819462,0.816327,0.812183,0.820513,0.807494,0.807692,0.535957,0.640854,0.738126,0.710526,0.637584,0.795892,0.820611,0.807062,0.826972,0.860349,0.835427,0.869565,0.837596,0.871212,0.826923,0.871212,0.844981,0.864041,0.833333,0.885678,0.864662,0.885678,0.842912,0.869289,0.851334,0.869018,0.892634,0.78125,0.815006,0.866584,0.890411,0.874214,0.85335,0.87469,0.878635,0.895522,0.8875,0.868188,0.883838,0.802872,0.678733,0.578313,0.726121,0.72318,0.737898,0.0078,0.562169,0.780409,0.621693,0.802616,0.751592,0.698433,0.787879,0.788486,0.714286,0.813665,0.789141,0.801001,0.825,0.816708,0.844636,0.806045,0.852479,0.852905,0.802781,0.846626,0.830189,0.843521,0.815287,0.829114,0.860149,0.822704,0.866337,0.875912,0.822704,0.832282,0.855746,0.833333,0.807494,0.840708,0.273312,0.47014,0.689441,0.726928,0.742386,0.808642,0.742638,0.801242,0.808271,0.795455,0.825062,0.789809,0.815558,0.802548,0.814536,0.827068,0.826033,0.846824,0.822785,0.831234,0.82702,0.833333,0.833333,0.829114,0.832282,0.82702,0.834387,0.836502,0.83014,0.849057,0.838588,0.842767,0.844893,0.822704,0.824808,0.83014,0.837563,0.840708,0.836502,0.836526,0.789474,0.812121,0.780457,0.77474,0.806658,0.81749,0.833333,0.840652,0.796632,0.801034,0.849515,0.840708,0.838588,0.815287,0.823755,0.826923,0.83014,0.841772,0.831202,0.850254,0.814249,0.855346,0.852273,0.84799,0.860553,0.848101,0.834395,0.858396,0.789133,0.835476,0.864428,0.865992,0.871446,0.87202,0.821566,0.839695,0.832266,0.852273,0.78776,0.850254,0.599078,0.550062,0.38036,0.530201,0.629096,0.794504,0.787841,0.813953,0.837079,0.81864,0.830235,0.796915,0.86747,0.86165,0.860149,0.815287,0.842839,0.762712,0.857143,0.85625,0.871951,0.8625,0.789133,0.797941,0.881273,0.882353,0.834403,0.888199,0.835476,0.878049,0.734127,0.853503,0.837596,0.880893,0.874214,0.886792,0.872611,0.85443,0.844893,0.86514])[::-1]

array([0.910758, 0.895522, 0.892634, ..., 0.      , 0.      , 0.      ])

In [14]:
tf.keras.backend.clear_session()

try:
    del model
    del Model
    print("Old Model deleted.")
except(NameError):
    print("No Model yet.")
    pass

tf.keras.backend.clear_session()     

#For tensorflow.keras dropout rate, is a value between 0 and 1. Fraction of the input units to drop. 
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, input_shape=(127,),activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(400,activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(500,activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(600,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])
model.summary()

No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               25600     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               60300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               120400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)  

In [20]:
# Getting the weight that got us the highest score
model.load_weights("median_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.001/checkpoint_20201112-001113_Epoch_037")

In [21]:
x_test=new_standard_test_median_all_features_df.to_numpy()
y_test = pd.get_dummies(y_test_median_all_features).values

In [22]:
y_pred=np.round(model.predict(x_test))

In [23]:
f2_score=np.round(fbeta_score(y_test[:,1], y_pred[:,1], beta=2,average="binary",pos_label=1),6)
f2_score

0.894584

In [14]:
# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    
    true = pd.Series(list(Y_true))
    predicted = pd.Series(list(Y_pred))

    return pd.crosstab(true, predicted, rownames=['True'], colnames=['Predicted'])

In [25]:
cm = confusion_matrix(y_test[:,1], y_pred[:,1])
cm

Predicted,0.0,1.0
True,,
0,11751,49
1,15,185


**Trying with the highest adasyn data weights**

In [34]:
#adasyn
np.sort([0.914496,0.924599,0.930197,0.932584,0.933419,0.934164,0.93466,0.934558,0.934519,0.935788,0.935768,0.936444,0.936786,0.936424,0.936962,0.937509,0.93666,0.936412,0.935474,0.935499,0.935107,0.935488,0.935771,0.936037,0.935761,0.936507,0.936568,0.936781,0.936568,0.936124,0.936159,0.93663,0.936595,0.936781,0.937642,0.936772,0.93647,0.937109,0.937207,0.936887,0.931311,0.93425,0.93336,0.936825,0.934438,0.934377,0.93493,0.937624,0.940323,0.93694,0.939799,0.939669,0.938825,0.937278,0.938096,0.938203,0.936904,0.942128,0.941576,0.941141,0.939332,0.943506,0.936886,0.936093,0.93886,0.942903,0.937349,0.938069,0.938372,0.942144,0.938247,0.941774,0.938229,0.941419,0.93596,0.941753,0.940291,0.936895,0.942587,0.938683,0.923037,0.936005,0.939472,0.942365,0.933538,0.937616,0.939997,0.934474,0.933804,0.941212,0.941384,0.934595,0.937544,0.937829,0.932099,0.94263,0.936164,0.943386,0.935728,0.934204,0.948818,0.948369,0.941308,0.946919,0.940821,0.939803,0.936512,0.938754,0.941984,0.936806,0.764695,0.931178,0.936797,0.940636,0.936112,0.901205,0.938229,0.948366,0.945911,0.936441,0.425295,0.666296,0.743269,0.79105,0.872664,0.904938,0.916103,0.92026,0.922105,0.925438,0.9293,0.931476,0.933617,0.935733,0.937439,0.941546,0.945365,0.94842,0.949715,0.951517,0.95292,0.954464,0.955638,0.956619,0.957572,0.959031,0.961022,0.962443,0.963629,0.964546,0.966137,0.96734,0.967701,0.968488,0.969239,0.970231,0.970769,0.972016,0.97333,0.973423,0.935215,0.937908,0.943461,0.955993,0.98019,0.986015,0.989112,0.991099,0.994001,0.994562,0.995827,0.996787,0.997379,0.997604,0.998111,0.998279,0.998702,0.998533,0.998815,0.99873,0.998702,0.998815,0.999238,0.999125,0.999153,0.999238,0.999153,0.99921,0.999181,0.999322,0.999322,0.999294,0.999351,0.999181,0.999435,0.999576,0.999463,0.999492,0.999548,0.999605,0.943939,0.983252,0.988553,0.992204,0.992088,0.996701,0.991599,0.995084,0.99695,0.989384,0.997995,0.998758,0.997655,0.998616,0.997118,0.998447,0.996552,0.992019,0.99904,0.998277,0.99904,0.99952,0.997909,0.998389,0.999435,0.998898,0.998899,0.99952,0.998955,0.999096,0.999153,0.999463,0.999068,0.999209,0.999576,0.99887,0.998926,0.998785,0.997937,0.99952,0.846899,0.905584,0.916646,0.922006,0.926413,0.930324,0.933444,0.93821,0.941005,0.942192,0.943868,0.944504,0.945418,0.946515,0.947852,0.948794,0.949556,0.94991,0.950578,0.951021,0.951791,0.952803,0.953031,0.953815,0.95403,0.954371,0.954519,0.955591,0.956153,0.957345,0.957647,0.957842,0.958135,0.958605,0.958925,0.960175,0.960825,0.960879,0.961171,0.961652,0.944802,0.957361,0.962963,0.965467,0.968064,0.969834,0.971714,0.973152,0.974456,0.97834,0.980251,0.980618,0.982726,0.98366,0.98426,0.984524,0.986881,0.986796,0.98705,0.989187,0.987871,0.991078,0.990798,0.991947,0.992623,0.992367,0.993019,0.993271,0.993547,0.993356,0.993917,0.994195,0.994031,0.994114,0.994731,0.99569,0.995293,0.994987,0.996111,0.996565,0.966473,0.976511,0.983265,0.988962,0.990084,0.990631,0.989121,0.994685,0.995836,0.995494,0.995723,0.996424,0.996509,0.997379,0.996846,0.997323,0.997436,0.998083,0.997914,0.997576,0.997351,0.997464,0.997885,0.997492,0.997632,0.997942,0.998336,0.997745,0.997829,0.998449,0.997998,0.997998,0.99842,0.998167,0.998026,0.997801,0.998336,0.998336,0.998702,0.998449,0.875095,0.913183,0.921796,0.926589,0.930175,0.933106,0.935589,0.938039,0.940749,0.943296,0.944871,0.94652,0.94776,0.948737,0.949861,0.950854,0.953128,0.954817,0.955291,0.956788,0.957703,0.958954,0.959855,0.960292,0.960756,0.961484,0.961808,0.962678,0.962991,0.963634,0.964272,0.964587,0.965004,0.965457,0.966291,0.966481,0.967184,0.968114,0.968726,0.969008,0.941873,0.956816,0.964713,0.965558,0.970987,0.973352,0.975699,0.977781,0.977793,0.980776,0.981653,0.981733,0.984855,0.983512,0.986076,0.986825,0.98792,0.988143,0.988853,0.989437,0.99011,0.991156,0.989826,0.991944,0.993123,0.992,0.993011,0.994928,0.996,0.996,0.996309,0.995545,0.996252,0.99707,0.997155,0.99752,0.997857,0.997351,0.997323,0.997407,0.965722,0.980775,0.989185,0.989166,0.990506,0.992256,0.992896,0.994897,0.994762,0.996873,0.997295,0.997211,0.997322,0.997576,0.997351,0.997717,0.998336,0.997716,0.997886,0.998448,0.997829,0.998533,0.998194,0.996781,0.997829,0.998786,0.998561,0.998561,0.998533,0.997801,0.998787,0.998758,0.999097,0.998195,0.998787,0.998674,0.998927,0.998928,0.999125,0.998645,0.902265,0.91024,0.914202,0.91716,0.920315,0.922433,0.922845,0.925165,0.9264,0.928464,0.929532,0.93083,0.931255,0.932614,0.933591,0.934288,0.934472,0.935383,0.935953,0.935336,0.936154,0.936875,0.937553,0.937016,0.937077,0.937394,0.937791,0.9381,0.937994,0.938584,0.938426,0.938752,0.938648,0.93885,0.939088,0.938957,0.93885,0.938718,0.938639,0.938542,0.932286,0.939202,0.938214,0.938392,0.938375,0.937925,0.940251,0.939397,0.939831,0.939184,0.938059,0.935972,0.936737,0.939115,0.94003,0.938646,0.93861,0.938743,0.940879,0.938619,0.937731,0.93661,0.936655,0.937784,0.938602,0.939233,0.937793,0.938993,0.939658,0.9379,0.940979,0.938451,0.936343,0.939926,0.939429,0.937224,0.935157,0.940041,0.940982,0.939811,0.932716,0.934726,0.925354,0.940626,0.938611,0.9326,0.944528,0.942601,0.94075,0.938948,0.936049,0.945149,0.93814,0.933829,0.929754,0.939029,0.931645,0.941741,0.945417,0.939065,0.93073,0.931766,0.932786,0.951896,0.93669,0.942165,0.93893,0.943118,0.929259,0.919967,0.942247,0.945406,0.949555,0.940493,0.901438,0.925026,0.930243,0.93605,0.921776,0.93653,0.861807,0.901534,0.890989,0.866803,0.84567,0.826958,0.826608,0.819842,0.81453,0.818675,0.827097,0.833576,0.842134,0.849259,0.859181,0.864247,0.869121,0.874755,0.877584,0.888373,0.892898,0.895883,0.89861,0.900289,0.90327,0.906546,0.90826,0.910998,0.912385,0.914146,0.916308,0.917978,0.9191,0.921356,0.923698,0.924525,0.92631,0.928037,0.930305,0.931498,0.794035,0.818914,0.860001,0.900618,0.905445,0.898271,0.915335,0.905676,0.915827,0.913276,0.918784,0.924071,0.933454,0.933156,0.931396,0.940379,0.948702,0.953753,0.959911,0.958314,0.96614,0.970694,0.961812,0.969765,0.962728,0.973284,0.972451,0.973339,0.968926,0.971219,0.973537,0.975281,0.981076,0.975308,0.979421,0.986143,0.981444,0.983978,0.986257,0.986596,0.872041,0.919614,0.882771,0.925591,0.953502,0.943294,0.967407,0.966797,0.958144,0.970888,0.968659,0.970026,0.981913,0.960804,0.969697,0.974113,0.975288,0.981023,0.977767,0.979282,0.979254,0.988255,0.993187,0.985718,0.990549,0.989927,0.994093,0.989899,0.98814,0.991656,0.991544,0.994462,0.997573,0.993132,0.991796,0.994827,0.991657,0.996498,0.993611,0.987198,0.900298,0.910384,0.916277,0.920559,0.923045,0.925784,0.928894,0.932958,0.935014,0.937259,0.938406,0.939383,0.940674,0.941646,0.942654,0.944754,0.945953,0.947412,0.947769,0.949234,0.949615,0.950263,0.950785,0.951374,0.951897,0.952626,0.953633,0.953912,0.954781,0.955145,0.955444,0.955882,0.956208,0.956745,0.957396,0.957637,0.95823,0.958278,0.958652,0.958979,0.933579,0.945969,0.948302,0.952187,0.957103,0.958831,0.962608,0.962447,0.963718,0.964406,0.966055,0.966599,0.968059,0.96713,0.968404,0.968544,0.969143,0.969845,0.969719,0.970162,0.969927,0.970135,0.970277,0.971832,0.970778,0.970922,0.971828,0.972675,0.971953,0.971961,0.972101,0.972121,0.972156,0.973136,0.972937,0.973042,0.973292,0.974773,0.974086,0.975099,0.954583,0.963206,0.96771,0.970617,0.973692,0.981458,0.985805,0.985091,0.986751,0.986429,0.988671,0.988523,0.988117,0.989816,0.989534,0.992268,0.991009,0.990995,0.992916,0.99091,0.991537,0.993487,0.992815,0.992947,0.992176,0.994015,0.993474,0.992679,0.994937,0.993794,0.994575,0.994518,0.99449,0.995105,0.994036,0.993681,0.993765,0.994683,0.993153,0.993625,0.902849,0.914939,0.918323,0.921752,0.925946,0.929118,0.931565,0.933921,0.936337,0.93816,0.938951,0.940553,0.941801,0.942649,0.943323,0.944497,0.944772,0.945527,0.946309,0.9466,0.946657,0.94668,0.947093,0.947725,0.947991,0.948389,0.949163,0.949536,0.949648,0.950416,0.950956,0.951311,0.951652,0.952013,0.953305,0.95378,0.954236,0.954999,0.955448,0.955837,0.932117,0.943431,0.950179,0.953873,0.960371,0.961403,0.964037,0.964856,0.964344,0.96646,0.968064,0.968777,0.968878,0.970248,0.970685,0.970684,0.971388,0.97169,0.973236,0.972728,0.972778,0.974207,0.972955,0.974372,0.975032,0.975035,0.976029,0.975837,0.976308,0.97635,0.977059,0.978169,0.977433,0.977629,0.979064,0.979144,0.979515,0.981445,0.981246,0.981087,0.96184,0.969343,0.979688,0.985493,0.97904,0.987774,0.98792,0.988527,0.989255,0.989579,0.990699,0.99116,0.990708,0.990173,0.991439,0.991602,0.991623,0.991862,0.994173,0.993727,0.993475,0.993133,0.991583,0.992594,0.994573,0.993363,0.994283,0.994598,0.994563,0.995189,0.993952,0.993398,0.992317,0.991372,0.990241,0.991676,0.990695,0.990692,0.992188,0.989842,0.91741,0.925616,0.930994,0.934329,0.93763,0.939528,0.94063,0.940191,0.940143,0.940866,0.940641,0.940988,0.941481,0.942005,0.941743,0.940971,0.941702,0.941352,0.941346,0.941436,0.94223,0.942088,0.941939,0.941324,0.942135,0.941871,0.94122,0.941459,0.941812,0.941998,0.941832,0.942324,0.942398,0.942796,0.94255,0.94247,0.942478,0.942232,0.941393,0.94135,0.936755,0.943359,0.942549,0.943057,0.939467,0.942164,0.937943,0.939424,0.941195,0.940303,0.939897,0.938113,0.939347,0.939995,0.93901,0.939774,0.940005,0.9383,0.940148,0.940662,0.938896,0.938576,0.938122,0.940591,0.937447,0.938718,0.93726,0.942309,0.941708,0.938896,0.939641,0.935193,0.939278,0.938807,0.941442,0.939474,0.93958,0.941726,0.939055,0.936797,0.938897,0.933142,0.934789,0.942342,0.921896,0.927001,0.936139,0.937126,0.941051,0.941105,0.931195,0.943008,0.940832,0.936361,0.941461,0.944282,0.941866,0.947547,0.931935,0.942842,0.950122,0.950212,0.943434,0.940981,0.935925,0.945551,0.942885,0.937491,0.943495,0.939304,0.94716,0.945329,0.942523,0.939162,0.917343,0.947175,0.933288,0.943568,0.935204,0.92511,0.623113,0.481178,0.697664,0.822612,0.870526,0.895633,0.90085,0.905568,0.907231,0.910808,0.913053,0.91666,0.919195,0.921586,0.922322,0.922588,0.925133,0.925826,0.926547,0.927278,0.928927,0.931551,0.931502,0.933786,0.936559,0.936576,0.937153,0.937722,0.937962,0.939907,0.942099,0.943678,0.945044,0.946683,0.948462,0.949915,0.951159,0.951962,0.953893,0.95622,0.904077,0.928031,0.940916,0.954605,0.967646,0.977526,0.984068,0.988341,0.989914,0.991374,0.993118,0.99453,0.994556,0.995372,0.99622,0.996189,0.995313,0.99729,0.997799,0.997996,0.996583,0.997034,0.996695,0.996242,0.99757,0.996892,0.997853,0.998786,0.997966,0.997712,0.998672,0.996665,0.996948,0.997825,0.9974,0.99822,0.997683,0.997231,0.999124,0.997626,0.889746,0.932851,0.899208,0.957104,0.93835,0.967375,0.956995,0.976281,0.954379,0.970658,0.97455,0.968944,0.921628,0.924354,0.975655,0.965859,0.98208,0.989981,0.976084,0.962465,0.977687,0.944479,0.940241,0.988079,0.983811,0.989327,0.97651,0.919787,0.990207,0.986115,0.99205,0.994883,0.988815,0.967949,0.904621,0.995252,0.991313,0.980901,0.992871,0.970551,0.922103,0.930576,0.933982,0.936544,0.939085,0.941736,0.943533,0.945124,0.945893,0.946834,0.948679,0.949936,0.950588,0.952581,0.953961,0.955322,0.956216,0.956958,0.958082,0.959257,0.959871,0.959654,0.960432,0.960718,0.961247,0.961306,0.961657,0.962079,0.962327,0.962716,0.963554,0.963544,0.963943,0.964311,0.96436,0.964641,0.964782,0.965203,0.965765,0.966291,0.947729,0.956645,0.960233,0.963487,0.964769,0.966908,0.968471,0.969944,0.969979,0.970845,0.97134,0.971408,0.972136,0.97216,0.972296,0.971883,0.9731,0.975243,0.974356,0.975866,0.976134,0.979237,0.981278,0.982947,0.982772,0.983814,0.983947,0.984606,0.986733,0.987644,0.988732,0.989585,0.991567,0.991827,0.992136,0.993313,0.994239,0.994015,0.994323,0.994547,0.960672,0.968479,0.971145,0.982651,0.985667,0.986355,0.987649,0.988922,0.989569,0.989955,0.992954,0.991724,0.990855,0.993652,0.993287,0.993904,0.994045,0.994492,0.994743,0.993209,0.992872,0.99318,0.990896,0.993878,0.990565,0.990373,0.992786,0.980634,0.978186,0.973112,0.946824,0.957702,0.971342,0.970991,0.971985,0.973561,0.97371,0.976905,0.978867,0.975697,0.91803,0.925012,0.929856,0.933964,0.936438,0.939304,0.942498,0.944772,0.946509,0.947497,0.948814,0.949718,0.950251,0.950718,0.951773,0.952321,0.953304,0.953412,0.954397,0.9548,0.956387,0.957183,0.957085,0.957944,0.958642,0.960088,0.960543,0.961641,0.961955,0.961977,0.962475,0.962534,0.962957,0.963866,0.963747,0.964038,0.964093,0.965137,0.965347,0.965428,0.946478,0.961425,0.966304,0.968517,0.969771,0.971937,0.972609,0.974316,0.977275,0.977162,0.978445,0.983948,0.983163,0.985473,0.986968,0.98657,0.987324,0.989831,0.99127,0.99217,0.991972,0.992562,0.992868,0.993521,0.993663,0.994249,0.99631,0.996196,0.996338,0.995915,0.996703,0.996647,0.99676,0.997294,0.99735,0.997182,0.997239,0.99766,0.997688,0.99766,0.963056,0.980337,0.984244,0.986127,0.987243,0.986835,0.989218,0.989512,0.989711,0.990664,0.988601,0.991939,0.990623,0.989674,0.991007,0.992284,0.989738,0.991637,0.989843,0.991395,0.989178,0.989452,0.989846,0.983705,0.985847,0.986736,0.979326,0.979166,0.977594,0.97873,0.9804,0.977073,0.980166,0.983474,0.984416,0.986576,0.983322,0.984395,0.985825,0.986171,0.913004,0.920452,0.92726,0.934451,0.93421,0.936652,0.93883,0.940236,0.941537,0.940892,0.94166,0.941657,0.941659,0.941943,0.9417,0.942544,0.942778,0.943085,0.943852,0.943572,0.943556,0.943204,0.943354,0.943854,0.943324,0.94382,0.944017,0.94438,0.944771,0.944448,0.944536,0.945266,0.944876,0.944378,0.945258,0.945422,0.946019,0.945457,0.946764,0.945051,0.937193,0.94446,0.943175,0.941295,0.94034,0.9445,0.945119,0.942068,0.940031,0.935685,0.942831,0.938655,0.940777,0.940438,0.940874,0.941026,0.942091,0.939846,0.941895,0.944263,0.937144,0.941877,0.935747,0.93822,0.943241,0.942064,0.941327,0.941401,0.9371,0.943289,0.940342,0.941452,0.941656,0.939465,0.944178,0.937313,0.934838,0.938434,0.944457,0.941807,0.941906,0.940766,0.941449,0.934074,0.939732,0.938699,0.95335,0.93011,0.924903,0.57517,0.944811,0.931013,0.945229,0.941568,0.905296,0.942118,0.938656,0.934694,0.911559,0.945152,0.936388,0.939945,0.940263,0.936459,0.933424,0.94263,0.946145,0.958125,0.946237,0.937624,0.950222,0.936041,0.94927,0.930166,0.949048,0.94367,0.928994,0.946084,0.938602,0.9429,0.721532,0.833564,0.82331,0.876797,0.894136,0.903145,0.906883,0.911311,0.919501,0.924557,0.930108,0.932251,0.932936,0.935088,0.938127,0.940647,0.94268,0.948156,0.950326,0.956533,0.957994,0.960256,0.963549,0.9671,0.969709,0.972469,0.976692,0.978417,0.979795,0.981859,0.983269,0.984419,0.9851,0.986507,0.986594,0.987322,0.987689,0.988194,0.988416,0.989378,0.905962,0.944239,0.962696,0.986977,0.989988,0.992993,0.994503,0.996731,0.997068,0.997801,0.998139,0.998279,0.998223,0.998505,0.998589,0.998561,0.998871,0.99873,0.99904,0.998928,0.999322,0.999238,0.999294,0.999069,0.99921,0.999379,0.999294,0.999322,0.999181,0.999746,0.999379,0.999463,0.999576,0.999605,0.999661,0.999435,0.999718,0.999689,0.999351,0.999492,0.948019,0.966516,0.98753,0.982441,0.995611,0.993189,0.994908,0.992493,0.991926,0.992636,0.995409,0.995381,0.994684,0.993357,0.993289,0.995887,0.995245,0.994936,0.99516,0.985357,0.985435,0.988241,0.984518,0.979548,0.983388,0.989931,0.985372,0.982708,0.985165,0.976461,0.97793,0.980498,0.981819,0.982049,0.974747,0.981605,0.98243,0.986754,0.986901,0.981722,0.907146,0.924136,0.936333,0.940822,0.944075,0.94899,0.95199,0.953118,0.955175,0.956317,0.958616,0.960331,0.961173,0.961818,0.962591,0.963012,0.96413,0.96489,0.965648,0.966862,0.967448,0.967471,0.967918,0.968627,0.968663,0.969819,0.970172,0.970609,0.970978,0.970874,0.972642,0.971756,0.974113,0.973152,0.973759,0.974295,0.975436,0.97521,0.976976,0.978005,0.952527,0.963187,0.966123,0.969997,0.971546,0.972661,0.977586,0.979764,0.980436,0.983221,0.984726,0.984632,0.987677,0.98734,0.989445,0.989389,0.990063,0.991277,0.992875,0.99268,0.991524,0.993572,0.994437,0.993989,0.994432,0.993339,0.994827,0.995192,0.995331,0.995276,0.995248,0.99578,0.99606,0.996313,0.996032,0.996313,0.996285,0.996481,0.996509,0.997014,0.968356,0.976273,0.985872,0.989891,0.992233,0.993682,0.99533,0.994267,0.994937,0.99578,0.995045,0.99442,0.996986,0.996312,0.997098,0.996446,0.992845,0.997042,0.997041,0.997744,0.997266,0.997433,0.997351,0.996818,0.99797,0.998364,0.996196,0.998026,0.997406,0.993934,0.987997,0.977735,0.951351,0.95038,0.952195,0.91486,0.942635,0.951632,0.936277,0.921865,0.910177,0.924767,0.93083,0.934853,0.938931,0.939889,0.942212,0.945548,0.948632,0.949088,0.950505,0.951402,0.952676,0.953159,0.953857,0.955267,0.958547,0.959001,0.959966,0.960904,0.960947,0.961853,0.962417,0.963473,0.964796,0.965555,0.966687,0.96776,0.968937,0.969296,0.969527,0.970019,0.971326,0.971874,0.971753,0.972179,0.972805,0.972582,0.973465,0.973997,0.950853,0.962524,0.968343,0.976744,0.975039,0.981259,0.982128,0.985133,0.985046,0.988235,0.9909,0.984989,0.988758,0.989661,0.994155,0.994968,0.994176,0.993642,0.993837,0.994487,0.995243,0.995948,0.993861,0.996453,0.996054,0.996227,0.996397,0.996874,0.996649,0.997295,0.996789,0.99707,0.997632,0.997323,0.997379,0.99752,0.997914,0.998025,0.998026,0.99752,0.970718,0.986018,0.990249,0.992565,0.993752,0.995044,0.995521,0.996705,0.995135,0.996531,0.99752,0.996929,0.998251,0.9978,0.998195,0.998843,0.998815,0.998476,0.998815,0.99873,0.998618,0.998674,0.998815,0.998758,0.998449,0.998111,0.996313,0.993544,0.993655,0.924031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.895387,0.916689,0.9217,0.923866,0.925437,0.926416,0.927032,0.928226,0.928744,0.929727,0.930236,0.930903,0.931022,0.931121,0.93107,0.930758,0.930968,0.93153,0.931693,0.931796,0.932244,0.93258,0.933137,0.933152,0.933097,0.932857,0.932528,0.932126,0.932331,0.931919,0.932016,0.932406,0.932663,0.933169,0.933229,0.933309,0.933352,0.933494,0.934046,0.934036,0.9195,0.931964,0.931737,0.934233,0.935006,0.934647,0.937864,0.936593,0.937473,0.938388,0.93894,0.937607,0.93718,0.940217,0.938362,0.9395,0.94075,0.941034,0.940688,0.937313,0.937642,0.940342,0.937874,0.939606,0.937607,0.93822,0.939518,0.941178,0.938717,0.941533,0.939527,0.940148,0.93637,0.94179,0.939145,0.936583,0.941187,0.936209,0.937705,0.940362,0.927653,0.929921,0.939836,0.933742,0.927831,0.930136,0.946455,0.938149,0.945475,0.945441,0.940735,0.932266,0.941176,0.941568,0.933347,0.927673,0.950068,0.943929,0.929967,0.93958,0.938478,0.939624,0.94557,0.924893,0.945486,0.934072,0.939242,0.931623,0.932392,0.93497,0.932286,0.938665,0.949459,0.925661,0.94631,0.936886,0.937642,0.924681,0.937109,0.936379,0.700375,0.835306,0.880947,0.900328,0.905378,0.910935,0.913652,0.917467,0.919674,0.922171,0.924091,0.926548,0.928381,0.930595,0.931837,0.933782,0.935079,0.936482,0.938659,0.939765,0.940556,0.941479,0.942736,0.943388,0.943699,0.944946,0.945798,0.94633,0.948664,0.950446,0.95151,0.954369,0.955796,0.956379,0.957809,0.95916,0.960252,0.961895,0.962965,0.963927,0.922678,0.920223,0.928905,0.931296,0.923618,0.923189,0.930954,0.92835,0.933843,0.937268,0.941673,0.948977,0.951899,0.954676,0.962447,0.969772,0.970588,0.975426,0.976963,0.976901,0.984733,0.987686,0.987144,0.991092,0.991655,0.992279,0.993669,0.993016,0.992787,0.993864,0.99409,0.994487,0.993892,0.995138,0.994572,0.995364,0.996524,0.998418,0.998192,0.997739,0.94297,0.965155,0.984159,0.988972,0.99125,0.993506,0.99608,0.996248,0.993843,0.99605,0.992314,0.997094,0.997348,0.997773,0.998364,0.997884,0.997347,0.996612,0.997997,0.997969,0.996979,0.997545,0.998674,0.997688,0.997318,0.998645,0.997487,0.997572,0.998589,0.99921,0.998702,0.99791,0.998221,0.99808,0.998024,0.998588,0.997232,0.998532,0.998814,0.999153,0.882217,0.912687,0.925794,0.932108,0.935593,0.936834,0.938566,0.941248,0.943669,0.944761,0.946217,0.94642,0.947075,0.948086,0.948641,0.949,0.949715,0.950087,0.950471,0.953386,0.953566,0.95515,0.955843,0.95617,0.956658,0.956872,0.95742,0.95835,0.958764,0.959512,0.960412,0.960708,0.960724,0.96128,0.961943,0.962477,0.962865,0.963114,0.963157,0.963524,0.941411,0.952054,0.958442,0.963062,0.965689,0.967443,0.968513,0.969855,0.970664,0.970831,0.972269,0.97341,0.973759,0.974484,0.975733,0.976756,0.976651,0.978163,0.980171,0.978892,0.978849,0.979367,0.980392,0.980718,0.982186,0.981559,0.980774,0.98202,0.981642,0.981979,0.983362,0.984321,0.984464,0.986466,0.986753,0.985054,0.987085,0.9876,0.988947,0.987961,0.964136,0.971396,0.971571,0.976029,0.977574,0.986062,0.990342,0.99147,0.993503,0.993439,0.994677,0.99547,0.99679,0.996648,0.996844,0.996367,0.997211,0.996902,0.996817,0.997575,0.997408,0.99693,0.997492,0.997211,0.997267,0.997689,0.998223,0.997998,0.997857,0.99797,0.998111,0.998364,0.998223,0.998083,0.998195,0.998505,0.998139,0.99842,0.998392,0.997942,0.861581,0.892262,0.910453,0.921148,0.925148,0.92912,0.931085,0.932493,0.933696,0.936588,0.938948,0.940038,0.941752,0.943659,0.945551,0.94774,0.948751,0.950512,0.951563,0.952609,0.952957,0.953903,0.954641,0.954627,0.955487,0.956266,0.956829,0.958798,0.959984,0.961344,0.962069,0.963196,0.963559,0.964154,0.964668,0.96548,0.965604,0.96653,0.966779,0.967261,0.931854,0.949822,0.959978,0.964609,0.966726,0.969145,0.972105,0.97261,0.974147,0.974833,0.976372,0.977492,0.977829,0.979774,0.980784,0.982176,0.982231,0.983939,0.982699,0.984364,0.985576,0.986844,0.986985,0.986731,0.987715,0.987971,0.9889,0.989376,0.99059,0.989492,0.990169,0.991067,0.99177,0.991658,0.991995,0.9929,0.995041,0.993967,0.99504,0.995773,0.969694,0.981048,0.985872,0.990401,0.991582,0.993158,0.993861,0.994175,0.994876,0.995968,0.997265,0.997828,0.996874,0.997576,0.997518,0.997349,0.998223,0.998054,0.998223,0.998308,0.998392,0.998083,0.998308,0.997998,0.998646,0.998674,0.998646,0.998787,0.998646,0.998477,0.998645,0.998984,0.998758,0.998618,0.998561,0.998674,0.998871,0.99873,0.999125,0.999012,0.937143,0.93895,0.938066,0.939898,0.941225,0.939592,0.940503,0.942736,0.941826,0.940552,0.939915,0.941564,0.940784,0.94213,0.936771,0.942813,0.94128,0.941652,0.940306,0.937571,0.936878,0.941714,0.939179,0.940572,0.937144,0.938984,0.940252,0.94123,0.940041,0.940051,0.940227,0.939668,0.940289,0.940235,0.939437,0.937616,0.939357,0.940386,0.938717,0.939509,0.929869,0.943444,0.939516,0.939043,0.941854,0.934562,0.940939,0.939721,0.943621,0.934338,0.923484,0.941815,0.940103,0.940814,0.939535,0.947115,0.943249,0.939829,0.9418,0.928322,0.941763,0.935205,0.945041,0.938469,0.940583,0.939846,0.948234,0.936353,0.94705,0.942844,0.930657,0.942635,0.936468,0.918552,0.945178,0.950448,0.942595,0.941471,0.943485,0.944599,0.946361,0.944364,0.943858,0.944371,0.944848,0.943143,0.943527,0.942245,0.941335,0.942228,0.937926,0.938281,0.93663,0.939372,0.937899,0.937154,0.937074,0.935473,0.935713,0.938246,0.934707,0.936638,0.936362,0.9367,0.93758,0.936824,0.935641,0.935658,0.934937,0.937873,0.936424,0.935479,0.936326,0.937011,0.938895,0.935881,0.936085,0.936022,0.937945,0.936192,0.918562,0.940576,0.948699,0.963414,0.980406,0.986875,0.989798,0.991673,0.992941,0.994565,0.995526,0.996536,0.996761,0.997042,0.997266,0.99752,0.998139,0.998167,0.998223,0.998308,0.998392,0.998533,0.998477,0.998561,0.998646,0.998618,0.998449,0.998956,0.999012,0.998871,0.998928,0.999097,0.999069,0.99921,0.99921,0.999322,0.999238,0.999238,0.999238,0.99921,0.979205,0.995864,0.997463,0.998561,0.998871,0.998449,0.999266,0.99904,0.999492,0.999492,0.998532,0.999351,0.999407,0.999633,0.999351,0.999181,0.999379,0.999322,0.999802,0.999407,0.99952,0.999548,0.999576,0.999831,0.999746,0.999379,0.99952,0.999294,0.999548,0.999576,0.999633,0.999689,0.999576,0.999718,0.999718,0.999718,0.999633,0.999576,0.999718,0.999746,0.979815,0.993234,0.996088,0.996368,0.994866,0.997041,0.99535,0.995172,0.997432,0.998364,0.998109,0.99707,0.997939,0.996648,0.996869,0.997154,0.998589,0.997884,0.996536,0.997211,0.998335,0.995332,0.998054,0.996505,0.996981,0.996697,0.997322,0.998589,0.99477,0.988788,0.993148,0.98048,0.986601,0.990177,0.979283,0.988903,0.99012,0.988461,0.962627,0.978722,0.949985,0.957384,0.964286,0.966275,0.96963,0.972469,0.97377,0.974947,0.976327,0.977823,0.978444,0.980108,0.981402,0.985104,0.985614,0.984405,0.987303,0.986987,0.987446,0.989833,0.990962,0.991834,0.992172,0.993296,0.994426,0.995016,0.996031,0.996199,0.996282,0.995775,0.996648,0.996422,0.997155,0.996874,0.997042,0.997042,0.997323,0.997267,0.997576,0.997407,0.966473,0.981718,0.98767,0.994707,0.994673,0.997689,0.99752,0.997829,0.998223,0.99636,0.998223,0.998392,0.998589,0.998758,0.99842,0.998392,0.998787,0.999097,0.998449,0.998787,0.998336,0.998899,0.998392,0.998984,0.998956,0.998928,0.998787,0.999181,0.998928,0.998928,0.998702,0.998984,0.998956,0.998871,0.999153,0.998814,0.998815,0.998956,0.999294,0.999125,0.957119,0.966644,0.973455,0.979247,0.981935,0.985434,0.987869,0.989427,0.990864,0.994153,0.994072,0.995388,0.995101,0.996593,0.996846,0.99679,0.996761,0.997042,0.997323,0.997155,0.997857,0.997801,0.998223,0.99828,0.99828,0.998195,0.998054,0.998477,0.99828,0.998392,0.998054,0.998871,0.99842,0.998843,0.998758,0.998928,0.998928,0.998815,0.998646,0.998533,0.945867,0.953895,0.962053,0.965273,0.971313,0.972438,0.974605,0.975489,0.977546,0.979831,0.983006,0.983128,0.984742,0.986319,0.987256,0.988607,0.989485,0.989876,0.991453,0.9923,0.993965,0.994784,0.994781,0.996193,0.99591,0.996588,0.997406,0.996757,0.998026,0.998307,0.997914,0.997744,0.998026,0.998139,0.998223,0.99828,0.998054,0.998195,0.998505,0.998561,0.971858,0.985975,0.987576,0.993201,0.997913,0.997714,0.998392,0.998167,0.998561,0.998364,0.998364,0.998505,0.998843,0.999097,0.999125,0.998843,0.99904,0.999012,0.998278,0.999266,0.998956,0.999097,0.998928,0.99904,0.998899,0.999379,0.999351,0.998758,0.99921,0.999322,0.999181,0.99952,0.999435,0.999012,0.999181,0.999576,0.999633,0.999831,0.999407,0.999492,0.95787,0.970739,0.980791,0.98551,0.99298,0.994932,0.995385,0.996676,0.99707,0.996732,0.996593,0.997914,0.997688,0.997462,0.998083,0.99811,0.998306,0.998532,0.999097,0.998984,0.999125,0.999181,0.998561,0.998843,0.998251,0.998334,0.998815,0.998561,0.998956,0.998702,0.999012,0.998758,0.999322,0.999379,0.99904,0.994316,0.997153,0.997857,0.996874,0.99766])[::-1]

array([0.999831, 0.999831, 0.999802, ..., 0.      , 0.      , 0.      ])

In [35]:
tf.keras.backend.clear_session()

try:
    del model
    del Model
    print("Old Model deleted.")
except(NameError):
    print("No Model yet.")
    pass

tf.keras.backend.clear_session()     

#For tensorflow.keras dropout rate, is a value between 0 and 1. Fraction of the input units to drop. 
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, input_shape=(144,),activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(400,activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(500,activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(600,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])
model.summary()

Old Model deleted.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               29000     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               60300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               120400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5

In [36]:
# Getting the weight that got us the highest score
model.load_weights("adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.001/checkpoint_20201112-020820_Epoch_023")

In [37]:
x_test=new_standard_test_adasyn_all_features_df.to_numpy()
y_test = pd.get_dummies(y_test_adasyn_all_features).values

In [38]:
y_pred=np.round(model.predict(x_test))

In [39]:
f2_score=np.round(fbeta_score(y_test[:,1], y_pred[:,1], beta=2,average="binary",pos_label=1),6)
f2_score

0.895522

In [40]:
cm = confusion_matrix(y_test[:,1], y_pred[:,1])
cm

Predicted,0.0,1.0
True,,
0,11775,25
1,20,180


**Trying with the highest smotetomek data weights**

In [12]:
#smotetomek
np.sort([0.937062,0.945431,0.952316,0.955862,0.958897,0.961068,0.962165,0.962907,0.963394,0.964247,0.96489,0.963989,0.963165,0.964199,0.96372,0.964814,0.964144,0.964324,0.964126,0.964311,0.964692,0.964779,0.964969,0.964759,0.964092,0.964174,0.964429,0.964652,0.965054,0.96488,0.965103,0.964659,0.964664,0.964562,0.964178,0.964106,0.963763,0.963405,0.964905,0.963286,0.958077,0.960238,0.962314,0.959733,0.963006,0.962512,0.960454,0.960885,0.959658,0.960841,0.960241,0.961593,0.960312,0.960432,0.961168,0.962432,0.961887,0.961778,0.961282,0.962552,0.963847,0.9621,0.957688,0.959689,0.9594,0.960868,0.960988,0.962149,0.96332,0.959394,0.962563,0.960955,0.963211,0.96271,0.960955,0.962073,0.960835,0.962797,0.964197,0.962416,0.957257,0.961947,0.95993,0.960022,0.96048,0.964572,0.953783,0.965141,0.962546,0.964597,0.963287,0.963146,0.965779,0.96364,0.964979,0.964403,0.961833,0.961097,0.960955,0.967455,0.966123,0.959394,0.966616,0.953789,0.961816,0.959662,0.965295,0.962182,0.964035,0.963386,0.965158,0.936579,0.947664,0.958142,0.960338,0.959219,0.950679,0.962699,0.959622,0.965615,0.892251,0.927235,0.941076,0.949522,0.951634,0.954843,0.95691,0.956808,0.957681,0.958668,0.959238,0.960309,0.96134,0.962038,0.963363,0.963984,0.964583,0.965471,0.966325,0.96718,0.967828,0.967651,0.968712,0.969379,0.97064,0.971128,0.97121,0.971503,0.971956,0.972023,0.972089,0.972715,0.972919,0.973455,0.973988,0.974562,0.974754,0.975029,0.97551,0.975706,0.941122,0.963718,0.971417,0.975486,0.979992,0.983789,0.985854,0.986655,0.989047,0.991527,0.992507,0.994284,0.994622,0.995351,0.995438,0.996478,0.996422,0.996899,0.996647,0.997463,0.997519,0.998055,0.997942,0.998139,0.998139,0.998392,0.998195,0.998111,0.99828,0.998336,0.998646,0.998392,0.998533,0.998533,0.998618,0.99859,0.998618,0.99873,0.998758,0.998843,0.967879,0.983935,0.990793,0.990496,0.994981,0.996109,0.997095,0.997071,0.997491,0.997715,0.997348,0.997377,0.997913,0.998025,0.99811,0.997799,0.998279,0.998026,0.998223,0.997911,0.998223,0.998589,0.998674,0.997883,0.997346,0.998674,0.99842,0.998617,0.998815,0.998814,0.998843,0.998786,0.998786,0.998786,0.998674,0.998702,0.999209,0.998589,0.998899,0.998391,0.917241,0.935908,0.944319,0.950648,0.955939,0.958571,0.960819,0.962568,0.963889,0.966939,0.968223,0.969005,0.969735,0.970462,0.971221,0.971243,0.971911,0.972043,0.972608,0.973072,0.974022,0.973762,0.97421,0.973256,0.973791,0.97369,0.973597,0.973706,0.973191,0.974104,0.973695,0.97386,0.973918,0.974227,0.974653,0.974879,0.975548,0.97563,0.975836,0.976426,0.964315,0.969979,0.972875,0.975538,0.977321,0.978252,0.979363,0.979648,0.979924,0.981067,0.981448,0.982125,0.982341,0.982806,0.983729,0.983699,0.98383,0.98507,0.985686,0.985735,0.986722,0.987262,0.986215,0.986801,0.987173,0.987524,0.988068,0.987929,0.989133,0.989249,0.989302,0.989667,0.98978,0.990174,0.991157,0.990957,0.990311,0.991889,0.990304,0.99158,0.972012,0.979334,0.981017,0.98506,0.98705,0.98866,0.98963,0.990796,0.99279,0.994164,0.99437,0.994564,0.994647,0.995127,0.995068,0.99543,0.996081,0.995574,0.995912,0.996561,0.996899,0.997265,0.997235,0.996815,0.99718,0.997266,0.997266,0.997408,0.997632,0.997772,0.99766,0.997715,0.997885,0.997885,0.99828,0.997491,0.998027,0.997913,0.998026,0.998308,0.920587,0.946482,0.951457,0.954791,0.956862,0.957672,0.958454,0.958319,0.959809,0.961243,0.962292,0.962358,0.963017,0.964789,0.965812,0.967266,0.968073,0.969362,0.969885,0.97042,0.970654,0.970986,0.97142,0.972066,0.971815,0.972379,0.972629,0.973169,0.972955,0.973295,0.973511,0.973629,0.974086,0.97435,0.974244,0.97453,0.974953,0.975469,0.97552,0.975775,0.960627,0.972755,0.97539,0.979292,0.982884,0.982766,0.98458,0.985759,0.987856,0.987839,0.989495,0.988754,0.988898,0.989488,0.989767,0.989401,0.990473,0.991545,0.991513,0.991826,0.993148,0.992048,0.992953,0.993655,0.993401,0.992842,0.993854,0.993656,0.99484,0.994414,0.994728,0.995065,0.995573,0.995487,0.995629,0.995376,0.995685,0.995825,0.995827,0.995826,0.980249,0.984426,0.98913,0.990691,0.99343,0.993856,0.994499,0.994699,0.995344,0.995287,0.995741,0.995938,0.996558,0.995966,0.996415,0.997829,0.997122,0.99712,0.997885,0.998111,0.998533,0.99842,0.998138,0.998027,0.998167,0.998138,0.998589,0.997883,0.998646,0.998505,0.998307,0.998336,0.998561,0.998308,0.998505,0.998055,0.998646,0.998617,0.998815,0.998674,0.935081,0.94193,0.945348,0.949183,0.951719,0.954709,0.956145,0.956868,0.957876,0.959246,0.960439,0.960893,0.961243,0.96177,0.962697,0.96382,0.963945,0.964097,0.964173,0.963964,0.964533,0.964836,0.965033,0.965347,0.965781,0.966058,0.965871,0.966307,0.966275,0.96612,0.966007,0.966266,0.966365,0.966484,0.966092,0.965887,0.965946,0.96631,0.966742,0.966591,0.96112,0.965235,0.966789,0.965202,0.965199,0.964478,0.964084,0.964421,0.964602,0.963438,0.964657,0.962502,0.963134,0.962649,0.962285,0.964227,0.962743,0.959297,0.962884,0.961822,0.959559,0.964771,0.960546,0.96119,0.963255,0.963309,0.961964,0.963576,0.962481,0.964147,0.962917,0.96412,0.961293,0.962274,0.962906,0.961985,0.963451,0.962939,0.961811,0.961816,0.955225,0.959005,0.962497,0.95366,0.965483,0.962094,0.959925,0.966497,0.966147,0.964222,0.952008,0.962824,0.960175,0.961669,0.959225,0.965274,0.951324,0.962399,0.96481,0.960709,0.961571,0.964021,0.96376,0.96186,0.960196,0.96525,0.964331,0.965748,0.963418,0.964294,0.954729,0.962198,0.962558,0.964137,0.952625,0.964925,0.961615,0.967633,0.960889,0.961664,0.724208,0.739287,0.761663,0.76378,0.766786,0.784328,0.796389,0.80527,0.833115,0.847013,0.862248,0.866836,0.871389,0.882615,0.886654,0.892498,0.900029,0.90244,0.904771,0.911546,0.916215,0.920771,0.923936,0.924511,0.925525,0.929391,0.93153,0.931395,0.934521,0.938362,0.939126,0.939926,0.941498,0.943662,0.941996,0.941775,0.945324,0.945343,0.946169,0.947835,0.925637,0.94693,0.958991,0.965649,0.969762,0.971051,0.972742,0.973595,0.975915,0.97772,0.977584,0.980892,0.981074,0.984755,0.985392,0.987559,0.98604,0.988886,0.989558,0.990348,0.991193,0.991891,0.993613,0.992627,0.992714,0.994175,0.99468,0.99395,0.993782,0.994425,0.995353,0.995325,0.9953,0.994963,0.995946,0.996424,0.996338,0.996508,0.996706,0.996508,0.898758,0.931805,0.935209,0.9527,0.920352,0.945077,0.936246,0.913299,0.926235,0.926756,0.92681,0.929942,0.944503,0.926943,0.93681,0.936236,0.949713,0.948296,0.946638,0.830512,0.929805,0.945807,0.909235,0.926411,0.858212,0.90482,0.919135,0.943504,0.956967,0.966149,0.810827,0.9283,0.94059,0.937043,0.84424,0.959707,0.925633,0.945988,0.94033,0.969623,0.935465,0.942266,0.947603,0.951936,0.955878,0.958817,0.960008,0.961351,0.962132,0.962968,0.963108,0.963308,0.963513,0.963714,0.963887,0.964346,0.965099,0.966041,0.966464,0.967299,0.968243,0.969618,0.96979,0.970545,0.970984,0.971852,0.972016,0.972183,0.972289,0.972429,0.973045,0.973475,0.973599,0.973716,0.973989,0.974061,0.974064,0.974205,0.974533,0.97442,0.959043,0.966727,0.971197,0.973363,0.974745,0.975034,0.973631,0.973971,0.975829,0.975998,0.974888,0.976513,0.976224,0.976816,0.97667,0.97756,0.977686,0.978273,0.978527,0.978475,0.977994,0.979051,0.978411,0.979553,0.979509,0.979897,0.980028,0.979591,0.981544,0.978525,0.977304,0.977404,0.978519,0.978749,0.97922,0.980345,0.976564,0.977456,0.979595,0.974975,0.969115,0.975668,0.978569,0.978215,0.97786,0.979225,0.977853,0.983141,0.982485,0.979845,0.980884,0.983505,0.985002,0.986853,0.990734,0.990127,0.986851,0.99115,0.991033,0.991759,0.991801,0.992599,0.992627,0.992655,0.992996,0.992715,0.992494,0.99116,0.990802,0.993027,0.99261,0.99341,0.993056,0.992321,0.991739,0.993755,0.993084,0.992783,0.992531,0.991255,0.919821,0.928589,0.93662,0.943919,0.946971,0.949433,0.951556,0.953602,0.955228,0.956531,0.957485,0.958137,0.959083,0.959941,0.960902,0.961292,0.96209,0.962841,0.962992,0.963975,0.964577,0.965589,0.966557,0.966967,0.967124,0.968225,0.968744,0.969182,0.969792,0.970082,0.970241,0.970373,0.970739,0.970817,0.971564,0.97186,0.971825,0.972047,0.972472,0.972936,0.95967,0.968326,0.971729,0.974895,0.975377,0.975078,0.975294,0.976823,0.976515,0.97794,0.97775,0.978074,0.979316,0.979368,0.980422,0.980991,0.982088,0.982105,0.982138,0.981871,0.983071,0.983572,0.983285,0.983771,0.984385,0.983722,0.984364,0.985094,0.983874,0.985222,0.984994,0.985636,0.985707,0.985546,0.986048,0.986644,0.98679,0.986557,0.987498,0.98795,0.969124,0.972723,0.978141,0.978509,0.982369,0.984142,0.984844,0.984752,0.986974,0.988027,0.989774,0.988918,0.990369,0.991858,0.992622,0.991825,0.993071,0.993517,0.993547,0.99371,0.994587,0.993725,0.992284,0.992674,0.994224,0.994333,0.994621,0.993553,0.993966,0.993082,0.992923,0.994261,0.994084,0.994447,0.993977,0.991914,0.992348,0.991939,0.991273,0.992117,0.937508,0.945769,0.951976,0.956237,0.95902,0.961389,0.963487,0.964636,0.965777,0.966646,0.966952,0.967737,0.968326,0.968258,0.968938,0.968898,0.968821,0.968985,0.968949,0.968967,0.968496,0.966788,0.96687,0.966778,0.966774,0.967315,0.967137,0.967537,0.967383,0.967528,0.96635,0.966145,0.966486,0.966108,0.965848,0.966244,0.966445,0.9667,0.966472,0.966153,0.962881,0.964061,0.967706,0.96613,0.967046,0.965044,0.964208,0.963475,0.965716,0.966269,0.966078,0.962328,0.9637,0.963618,0.962383,0.963482,0.96284,0.962045,0.963907,0.962999,0.96357,0.962727,0.962214,0.964299,0.963429,0.96265,0.964418,0.963821,0.963478,0.964838,0.962449,0.963794,0.9612,0.964935,0.9612,0.96253,0.958262,0.961577,0.964142,0.965322,0.956009,0.959251,0.954924,0.96254,0.959067,0.964348,0.96693,0.964674,0.964654,0.964435,0.962045,0.961391,0.963396,0.961124,0.963625,0.959219,0.958842,0.952313,0.964463,0.961429,0.966834,0.966643,0.965592,0.961435,0.967154,0.965533,0.963369,0.954,0.964632,0.962503,0.966798,0.967911,0.967655,0.967259,0.960507,0.966552,0.965446,0.962666,0.962901,0.946262,0.92856,0.938219,0.917452,0.906957,0.915221,0.91835,0.927519,0.935144,0.941208,0.944071,0.947056,0.949609,0.95068,0.953812,0.956273,0.956957,0.958739,0.959127,0.961501,0.961738,0.962734,0.963708,0.965324,0.966218,0.967093,0.967733,0.96801,0.968623,0.968783,0.969733,0.969878,0.970524,0.970738,0.970976,0.971631,0.971678,0.972152,0.972183,0.972195,0.972305,0.948233,0.961792,0.967097,0.969791,0.970993,0.974528,0.976497,0.982294,0.982281,0.985814,0.987905,0.989454,0.991829,0.991485,0.993404,0.992952,0.993826,0.995009,0.996052,0.996023,0.995937,0.996303,0.99667,0.997519,0.997658,0.997686,0.997969,0.998476,0.997798,0.997799,0.997091,0.99825,0.997911,0.998335,0.998617,0.998561,0.997628,0.998278,0.997882,0.998419,0.91228,0.95582,0.971731,0.979905,0.985722,0.985699,0.985492,0.9876,0.989936,0.991467,0.991561,0.991836,0.992885,0.993162,0.992772,0.990431,0.992464,0.992275,0.991134,0.992609,0.988902,0.992222,0.985464,0.991389,0.991804,0.990893,0.992314,0.990167,0.988663,0.989443,0.990858,0.985668,0.990332,0.988994,0.990554,0.989652,0.986256,0.986903,0.985906,0.987578,0.926877,0.945567,0.95352,0.957426,0.95986,0.961165,0.962774,0.964186,0.966721,0.967539,0.968589,0.969822,0.970446,0.971506,0.971837,0.972655,0.973029,0.973326,0.973822,0.974099,0.974369,0.974163,0.97405,0.97348,0.973352,0.973516,0.973942,0.973794,0.973423,0.973227,0.973306,0.97377,0.973649,0.973681,0.974044,0.974254,0.974463,0.974326,0.974319,0.97424,0.964716,0.973152,0.973527,0.975528,0.975401,0.976299,0.975967,0.977665,0.976698,0.977319,0.977309,0.979755,0.978426,0.97801,0.977695,0.978138,0.978861,0.978503,0.978534,0.978644,0.979076,0.978862,0.978442,0.978537,0.978819,0.979111,0.97931,0.979144,0.979096,0.980765,0.979399,0.983077,0.983156,0.983246,0.983944,0.985195,0.987513,0.986755,0.987854,0.988051,0.97449,0.975744,0.98094,0.982459,0.984419,0.987342,0.990004,0.990969,0.990656,0.990958,0.991374,0.992354,0.991965,0.991967,0.993161,0.993893,0.993474,0.993894,0.99409,0.993249,0.994425,0.994789,0.993691,0.993506,0.994067,0.994739,0.993895,0.994653,0.994773,0.994461,0.995638,0.995238,0.995778,0.995296,0.995807,0.994853,0.994521,0.992118,0.975926,0.965349,0.930427,0.945617,0.952564,0.956191,0.958885,0.959745,0.961159,0.962567,0.963367,0.964898,0.966967,0.96949,0.970525,0.971616,0.972292,0.972635,0.972207,0.97285,0.972854,0.973303,0.973694,0.973894,0.973918,0.974399,0.973938,0.974333,0.974275,0.974326,0.974645,0.974958,0.974659,0.974889,0.975033,0.97494,0.975246,0.976066,0.976121,0.975689,0.976372,0.976513,0.966521,0.974401,0.974835,0.975901,0.976471,0.977304,0.979487,0.980836,0.980549,0.982776,0.982864,0.982492,0.983129,0.98383,0.986193,0.984221,0.986207,0.986988,0.987769,0.988148,0.989607,0.990279,0.989017,0.99127,0.992428,0.991605,0.992676,0.993383,0.993774,0.994139,0.994731,0.994873,0.99507,0.995266,0.995434,0.9958,0.995715,0.995631,0.995491,0.99594,0.967629,0.967874,0.976275,0.982014,0.975903,0.984231,0.988038,0.990407,0.991759,0.991155,0.992426,0.993781,0.99208,0.992828,0.99319,0.994678,0.99111,0.991427,0.989408,0.989005,0.988095,0.989034,0.985224,0.990195,0.990442,0.989285,0.990013,0.990042,0.989744,0.99012,0.99061,0.990456,0.989621,0.990835,0.988265,0.99178,0.989866,0.991232,0.984867,0.989154,0.936773,0.948274,0.954573,0.959309,0.961977,0.964317,0.966587,0.967566,0.967117,0.967918,0.967302,0.967256,0.967415,0.966498,0.966592,0.966352,0.966287,0.96626,0.966556,0.96537,0.965608,0.967078,0.966323,0.966085,0.966263,0.965875,0.966289,0.966813,0.965893,0.966321,0.965821,0.965988,0.966549,0.967146,0.967014,0.966817,0.967218,0.967018,0.966671,0.967168,0.964838,0.966927,0.965058,0.966175,0.968061,0.967473,0.965626,0.963924,0.965771,0.963521,0.966255,0.96444,0.958585,0.963,0.961702,0.962737,0.963353,0.964753,0.96174,0.961566,0.963097,0.964299,0.962198,0.960606,0.964294,0.962525,0.961893,0.961806,0.964843,0.957782,0.962955,0.963483,0.963782,0.962258,0.959907,0.964681,0.962498,0.963439,0.960426,0.965213,0.965313,0.959083,0.961413,0.964297,0.964195,0.959656,0.961724,0.95621,0.966725,0.962595,0.966236,0.964696,0.959705,0.954249,0.950097,0.960524,0.96444,0.9652,0.964895,0.962743,0.947236,0.959625,0.960372,0.966862,0.96443,0.960551,0.928702,0.965089,0.958924,0.966802,0.947148,0.961162,0.964123,0.963843,0.966396,0.965694,0.964229,0.962247,0.963205,0.964957,0.894875,0.935244,0.937947,0.934682,0.934419,0.934236,0.931861,0.937642,0.942718,0.946249,0.949984,0.951854,0.955741,0.95754,0.958116,0.95962,0.961386,0.961816,0.96265,0.964383,0.964291,0.967185,0.967148,0.966965,0.966928,0.967472,0.968098,0.970122,0.970532,0.97098,0.971805,0.971825,0.972218,0.973987,0.974464,0.977237,0.979278,0.980326,0.981689,0.98259,0.954182,0.967255,0.968937,0.972371,0.979009,0.980631,0.990336,0.993439,0.993066,0.99496,0.995969,0.996588,0.997546,0.997407,0.998026,0.997321,0.998194,0.997998,0.99842,0.998702,0.998617,0.998759,0.998702,0.998956,0.999012,0.99873,0.998843,0.998984,0.999153,0.999012,0.999153,0.999097,0.999125,0.999322,0.999097,0.999181,0.999097,0.999294,0.999322,0.999351,0.869799,0.96186,0.954139,0.965799,0.969537,0.96317,0.974665,0.977734,0.984471,0.841677,0.96625,0.965801,0.976295,0.934612,0.942994,0.976408,0.977383,0.955419,0.972479,0.981358,0.983067,0.9767,0.978336,0.983241,0.977868,0.976292,0.974422,0.976665,0.978794,0.967537,0.973175,0.975403,0.982759,0.980493,0.980551,0.972583,0.975374,0.980648,0.971326,0.969349,0.950851,0.955254,0.955666,0.955475,0.955135,0.956236,0.958958,0.962717,0.961718,0.964055,0.965423,0.965463,0.966164,0.967332,0.965064,0.967464,0.968344,0.967427,0.9692,0.970052,0.970307,0.971163,0.971159,0.971214,0.970982,0.971693,0.972818,0.970432,0.972638,0.97343,0.97343,0.974343,0.974436,0.973454,0.974715,0.975486,0.974288,0.97455,0.975138,0.975627,0.969287,0.969221,0.973151,0.973084,0.975672,0.977646,0.976611,0.978393,0.980088,0.979978,0.980475,0.979466,0.981335,0.983747,0.983819,0.984401,0.984942,0.98516,0.986007,0.987055,0.98722,0.986999,0.987908,0.989155,0.989547,0.9892,0.990312,0.991694,0.991075,0.990793,0.992224,0.993071,0.992,0.992647,0.992309,0.993295,0.992393,0.992843,0.992676,0.993716,0.97692,0.982203,0.985732,0.987559,0.990954,0.993049,0.992746,0.994762,0.995576,0.995207,0.995887,0.996676,0.995443,0.996928,0.997351,0.996535,0.996814,0.996958,0.997858,0.997633,0.99749,0.997801,0.997998,0.997661,0.991649,0.965441,0.948241,0.959958,0.95217,0.946296,0.921683,0.907809,0.921145,0.799693,0.0,0.911338,0.937103,0.92794,0.928476,0.938793,0.934426,0.943694,0.951675,0.956492,0.959231,0.962388,0.964184,0.966211,0.96859,0.970095,0.971502,0.971587,0.972339,0.973077,0.974232,0.974108,0.973524,0.974677,0.974944,0.97456,0.975644,0.975414,0.97587,0.976303,0.97753,0.977836,0.978423,0.979049,0.97935,0.980103,0.98032,0.980381,0.980797,0.981209,0.980968,0.98155,0.981661,0.981523,0.98158,0.982055,0.968777,0.973722,0.977116,0.978923,0.980891,0.982389,0.984327,0.984404,0.986998,0.987312,0.987639,0.989759,0.990546,0.989311,0.991297,0.992764,0.992484,0.991975,0.992534,0.993209,0.993828,0.994928,0.995237,0.995886,0.994139,0.9958,0.99583,0.995547,0.995573,0.996335,0.995882,0.996392,0.996476,0.996365,0.996785,0.997039,0.996898,0.996955,0.997295,0.997433,0.978889,0.984484,0.987764,0.992659,0.994477,0.995155,0.995884,0.995995,0.996473,0.996983,0.997464,0.997659,0.997094,0.997462,0.998139,0.996312,0.993377,0.976094,0.953948,0.965999,0.963381,0.9466,0.951332,0.951562,0.955749,0.951485,0.952817,0.956808,0.96851,0.960834,0.958648,0.967055,0.95787,0.960208,0.955285,0.957714,0.957974,0.957455,0.95935,0.955646,0.919471,0.939056,0.945379,0.949425,0.952776,0.954798,0.956861,0.957497,0.959248,0.958829,0.959686,0.959975,0.959283,0.958952,0.959064,0.95944,0.959812,0.959475,0.959875,0.959651,0.960058,0.960317,0.960225,0.960567,0.961218,0.960827,0.960392,0.960897,0.960364,0.960821,0.960783,0.960755,0.960924,0.961038,0.959792,0.95971,0.959503,0.958941,0.959705,0.959622,0.959623,0.962345,0.963638,0.960079,0.962006,0.962839,0.963008,0.962596,0.963216,0.961974,0.962834,0.963864,0.961604,0.960011,0.962966,0.962296,0.96216,0.961582,0.963314,0.960938,0.962677,0.961086,0.96308,0.961348,0.964704,0.961866,0.960589,0.962966,0.962149,0.963048,0.962737,0.962007,0.958535,0.962846,0.962563,0.963277,0.961778,0.962558,0.964316,0.962443,0.954308,0.957767,0.955567,0.963864,0.955663,0.960438,0.963951,0.963494,0.959477,0.960328,0.965332,0.963919,0.964485,0.966711,0.966962,0.965277,0.962922,0.966168,0.950774,0.965583,0.965707,0.965684,0.960163,0.964224,0.966656,0.965986,0.960949,0.961593,0.953925,0.966145,0.956446,0.96663,0.964614,0.944161,0.962917,0.960884,0.954182,0.964371,0.965232,0.957755,0.82778,0.925272,0.941639,0.949777,0.953729,0.955793,0.957131,0.957414,0.957613,0.958002,0.959208,0.96034,0.960855,0.961937,0.962729,0.963227,0.96428,0.964654,0.964257,0.964873,0.966495,0.967216,0.967238,0.967424,0.968001,0.96855,0.969422,0.969763,0.969781,0.970008,0.970391,0.970532,0.970868,0.971095,0.971463,0.971858,0.971933,0.97239,0.972531,0.972614,0.954073,0.961158,0.967008,0.968531,0.97071,0.97359,0.973784,0.975689,0.979709,0.981891,0.984863,0.987035,0.987291,0.991833,0.991559,0.994119,0.994932,0.995523,0.99617,0.99617,0.996704,0.99707,0.996731,0.99752,0.997577,0.997745,0.997295,0.997998,0.9978,0.997914,0.997941,0.998364,0.998026,0.99828,0.998674,0.998335,0.998646,0.998759,0.998533,0.998702,0.966142,0.975665,0.983988,0.989428,0.990691,0.990545,0.992309,0.99176,0.995124,0.99543,0.996926,0.997519,0.995487,0.996785,0.994324,0.996388,0.996642,0.996812,0.996529,0.997602,0.997292,0.997686,0.995232,0.997207,0.996304,0.997292,0.997546,0.996162,0.997601,0.997855,0.998194,0.997432,0.997686,0.998025,0.997714,0.997969,0.99811,0.998081,0.998646,0.99828,0.865333,0.90924,0.931369,0.94134,0.946613,0.950253,0.952553,0.954701,0.95656,0.958458,0.959783,0.960901,0.961346,0.962214,0.962735,0.963465,0.964089,0.964858,0.966101,0.967259,0.968004,0.969461,0.970279,0.970826,0.971168,0.971802,0.972865,0.973585,0.973695,0.974163,0.974022,0.974245,0.974268,0.974323,0.974487,0.974733,0.974702,0.974781,0.974956,0.974661,0.963267,0.971585,0.974082,0.975034,0.975795,0.975527,0.977621,0.978756,0.979059,0.980285,0.980836,0.980914,0.981112,0.982472,0.981947,0.982346,0.982834,0.984031,0.984119,0.984114,0.984179,0.984834,0.984667,0.985626,0.985943,0.98563,0.986275,0.985761,0.986184,0.985906,0.986579,0.98635,0.987031,0.986495,0.987204,0.987828,0.988021,0.987454,0.988072,0.987482,0.976181,0.979532,0.981198,0.983376,0.983454,0.986279,0.985188,0.990708,0.990577,0.992249,0.992872,0.993265,0.992815,0.994338,0.994644,0.994732,0.994813,0.995038,0.994955,0.995407,0.995178,0.995855,0.99594,0.995714,0.995546,0.99591,0.996166,0.99656,0.996109,0.996363,0.996814,0.996785,0.997097,0.996872,0.997181,0.997349,0.997208,0.997068,0.996954,0.997744,0.917394,0.94075,0.947652,0.952552,0.955265,0.957208,0.959018,0.95982,0.96089,0.962585,0.963965,0.96429,0.965658,0.966966,0.968058,0.968546,0.969125,0.969704,0.97054,0.971735,0.972207,0.973115,0.973217,0.973123,0.973548,0.973584,0.973553,0.973827,0.974159,0.974066,0.974402,0.974313,0.974824,0.974591,0.974529,0.975236,0.975263,0.975061,0.975084,0.975163,0.962817,0.973444,0.975302,0.976973,0.979708,0.981052,0.982691,0.982952,0.983286,0.984279,0.985047,0.985267,0.985553,0.986447,0.986301,0.987042,0.98763,0.988075,0.988388,0.988472,0.9885,0.988636,0.988418,0.989511,0.989789,0.990409,0.990018,0.990409,0.991541,0.991374,0.990865,0.991768,0.991795,0.992808,0.993373,0.992615,0.993599,0.993346,0.993516,0.993488,0.975651,0.980764,0.982714,0.987117,0.98841,0.987039,0.98946,0.990014,0.992252,0.993092,0.995009,0.994108,0.995856,0.994701,0.995827,0.995655,0.996395,0.996786,0.996841,0.997575,0.99749,0.997348,0.998111,0.99811,0.997294,0.997206,0.998336,0.997941,0.997855,0.998081,0.998421,0.998392,0.99797,0.997998,0.997942,0.998195,0.998505,0.998364,0.998111,0.99859,0.963369,0.967276,0.968667,0.968806,0.969808,0.969707,0.964533,0.96554,0.963105,0.966902,0.968307,0.96735,0.965835,0.965903,0.962835,0.963362,0.964778,0.964485,0.963951,0.964788,0.963043,0.961626,0.964359,0.963913,0.96486,0.964892,0.96357,0.964495,0.963908,0.964197,0.964565,0.964942,0.962885,0.962165,0.96265,0.963674,0.963272,0.962765,0.96264,0.963782,0.961326,0.958852,0.965366,0.957742,0.962263,0.962034,0.96107,0.960475,0.960873,0.961337,0.958278,0.95994,0.963565,0.95964,0.943999,0.964686,0.963146,0.962433,0.958704,0.959693,0.958935,0.965039,0.96246,0.964654,0.964994,0.963309,0.959519,0.96518,0.954775,0.961457,0.956951,0.960878,0.964219,0.964147,0.949614,0.9632,0.965478,0.960442,0.958012,0.96253,0.883767,0.958524,0.961794,0.960271,0.955349,0.960069,0.962963,0.961131,0.965675,0.96367,0.962425,0.966207,0.968574,0.963474,0.964255,0.963225,0.964169,0.964625,0.96391,0.96347,0.963649,0.964349,0.963964,0.963997,0.963866,0.962709,0.965321,0.964259,0.962356,0.96333,0.96377,0.962715,0.963939,0.962159,0.964793,0.963644,0.960333,0.964173,0.962971,0.961151,0.954098,0.964767,0.970473,0.973696,0.975933,0.981119,0.984076,0.98808,0.992173,0.99296,0.993777,0.994478,0.995294,0.996027,0.996535,0.996957,0.997154,0.997491,0.997576,0.997604,0.997632,0.997576,0.997914,0.997914,0.998251,0.998083,0.998139,0.998251,0.998195,0.99842,0.998392,0.998589,0.998787,0.99873,0.998871,0.999012,0.998984,0.998871,0.998815,0.998787,0.974295,0.993546,0.996143,0.997913,0.998195,0.997942,0.998787,0.999294,0.998787,0.999153,0.999266,0.998843,0.999407,0.998956,0.998616,0.999351,0.999266,0.999435,0.99921,0.99952,0.999181,0.999407,0.999435,0.99952,0.999492,0.999407,0.99952,0.999379,0.999435,0.999633,0.999605,0.999463,0.999633,0.999661,0.999661,0.999492,0.999294,0.999548,0.999661,0.999689,0.971619,0.963955,0.994616,0.996845,0.996587,0.997576,0.995443,0.995051,0.996282,0.991084,0.994365,0.997829,0.997294,0.996227,0.990957,0.995156,0.996448,0.99631,0.994354,0.992199,0.994359,0.996364,0.993479,0.983036,0.970584,0.972725,0.987437,0.985943,0.989355,0.991551,0.984787,0.989188,0.97759,0.990648,0.990109,0.989443,0.988299,0.991517,0.989348,0.977935,0.971586,0.972085,0.97333,0.976743,0.978279,0.979779,0.980301,0.982055,0.982837,0.983577,0.984368,0.985094,0.985668,0.985689,0.986896,0.988649,0.988787,0.989177,0.989013,0.989327,0.989999,0.990815,0.990901,0.991546,0.991237,0.991996,0.993094,0.992928,0.993209,0.99411,0.993968,0.994164,0.994421,0.995039,0.995179,0.995293,0.99549,0.996082,0.995913,0.996111,0.979048,0.984253,0.986523,0.989943,0.994505,0.995183,0.997689,0.997012,0.997548,0.997715,0.997858,0.998505,0.998561,0.998083,0.998308,0.997577,0.998195,0.998336,0.998956,0.998759,0.998815,0.998928,0.998618,0.998702,0.99873,0.998674,0.999181,0.998871,0.998984,0.998843,0.99921,0.9989,0.999464,0.998871,0.99904,0.9989,0.99921,0.999181,0.999012,0.99904,0.945612,0.974792,0.979559,0.984188,0.984923,0.984904,0.993777,0.993923,0.994622,0.995638,0.99679,0.996165,0.997127,0.996956,0.997293,0.997183,0.997042,0.998026,0.997632,0.998336,0.997094,0.998279,0.998335,0.998533,0.998477,0.998251,0.997801,0.99873,0.998674,0.99873,0.998815,0.99856,0.998364,0.998843,0.999153,0.99726,0.998759,0.999097,0.99921,0.999125,0.965965,0.971037,0.975613,0.978046,0.980644,0.981022,0.98402,0.984861,0.985389,0.986573,0.987649,0.987811,0.988241,0.989112,0.989533,0.989563,0.990635,0.990947,0.991652,0.99278,0.991794,0.992979,0.993568,0.993288,0.993429,0.993317,0.994077,0.994755,0.994981,0.995206,0.995968,0.995433,0.995996,0.995968,0.996644,0.996532,0.997011,0.997011,0.997208,0.997406,0.982244,0.987799,0.989186,0.991929,0.993967,0.99594,0.997294,0.997321,0.997913,0.998448,0.99811,0.998561,0.997885,0.99794,0.998702,0.998589,0.998928,0.999153,0.99904,0.998759,0.998618,0.999125,0.999069,0.999012,0.998984,0.998984,0.9989,0.999125,0.999012,0.99921,0.999238,0.999435,0.999069,0.999407,0.99921,0.998984,0.9989,0.999238,0.999379,0.999548,0.977131,0.976011,0.983706,0.990153,0.99202,0.993198,0.993973,0.9962,0.995438,0.995835,0.997155,0.997605,0.996782,0.997548,0.998166,0.997997,0.99794,0.998448,0.998674,0.997886,0.992825,0.998533,0.998419,0.998815,0.998027,0.998871,0.998477,0.999125,0.999097,0.998702,0.999069,0.998843,0.999097,0.998336,0.998702,0.999294,0.998222,0.999153,0.998815,0.999125])[::-1]

array([0.999689, 0.999661, 0.999661, ..., 0.739287, 0.724208, 0.      ])

In [14]:
tf.keras.backend.clear_session()

try:
    del model
    del Model
    print("Old Model deleted.")
except(NameError):
    print("No Model yet.")
    pass

tf.keras.backend.clear_session()     

#For tensorflow.keras dropout rate, is a value between 0 and 1. Fraction of the input units to drop. 
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, input_shape=(117,),activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(400,activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(500,activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(600,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])
model.summary()

No Model yet.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               23600     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               60300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               120400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)  

In [16]:
# Getting the weight that got us the highest score
model.load_weights("smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.001/checkpoint_20201112-041629_Epoch_039")

In [17]:
x_test=new_standard_test_smotetomek_all_features_df.to_numpy()
y_test = pd.get_dummies(y_test_smotetomek_all_features).values

In [18]:
y_pred=np.round(model.predict(x_test))

In [19]:
f2_score=np.round(fbeta_score(y_test[:,1], y_pred[:,1], beta=2,average="binary",pos_label=1),6)
f2_score

0.910448

In [22]:
cm = confusion_matrix(y_test[:,1], y_pred[:,1])
cm

Predicted,0.0,1.0
True,,
0,11778,22
1,17,183


**Observation :**<br><br>
I observe that I get the highest score on cross validation on adasyn data.<br><br>
**But on the test data , I get the highest score on test on smotetomek data.**<br><br>
**<u>But There are a few configurations like : </u><br>
adasyn_data_checkpoint_400_0.8_200_0.4_100_0.6_50_0.2_10_0.5_selu_0.01/checkpoint_20201112-012718_Epoch_033,<br>
smotetomek_data_checkpoint_400_0.4_200_0.6_100_0.8_50_0.4_10_0.2_elu_0.01/checkpoint_20201112-035913_Epoch_028,<br>
smotetomek_data_checkpoint_400_0.4_200_0.6_100_0.8_50_0.4_10_0.2_elu_0.01/checkpoint_20201112-035913_Epoch_029,<br>
smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.01/checkpoint_20201112-041915_Epoch_024,<br>
smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.01/checkpoint_20201112-041915_Epoch_039,<br>
smotetomek_data_checkpoint_400_0.4_200_0.6_100_0.8_50_0.4_10_0.2_selu_0.01/checkpoint_20201112-035540_Epoch_025,<br>
smotetomek_data_checkpoint_400_0.4_200_0.6_100_0.8_50_0.4_10_0.2_elu_0.01/checkpoint_20201112-035913_Epoch_020<br>and many more that are giving me, <br><u>a perfect detection of downhole failures but at the cost of high misclassification of surface failures and a low f2 score.</u>**

In [20]:
try:
    del model
    del Model
    print("Old Model deleted.")
except(NameError):
    print("No Model yet.")
    pass

No Model yet.


In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, input_shape=(117,),activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(400,activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(500,activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(600,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

In [23]:
# Getting the weight that got us the highest score
model.load_weights("smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.01/checkpoint_20201112-041915_Epoch_039").expect_partial()

In [24]:
x_test=new_standard_test_smotetomek_all_features_df.to_numpy()
y_test = pd.get_dummies(y_test_smotetomek_all_features).values

In [25]:
x_test.shape
y_test.shape

(12000, 2)

In [26]:
y_pred=np.round(model.predict(x_test))

**Observe a low F2 Score**

In [27]:
f2_score=np.round(fbeta_score(y_test[:,1], y_pred[:,1], beta=2,average="binary",pos_label=1),6)
f2_score

0.52356

**Observe below perfect classification of downhole failures, but a huge misclassification of surface failures.**

In [29]:
cm = confusion_matrix(y_test[:,1], y_pred[:,1])
cm

Predicted,0.0,1.0
True,,
0,10890,910
1,0,200


**Conclusion : 
I will get a good F2 score only when there are both low downhole and low surface failures misclassification.<br>
Both are needed to be low. If I concentrate on removing the downhole misclassifications, then I observe a tradeoff between the downhole and surface misclassification.<br>
I tried running both adasyn test data and smotetomek test data but with 2 configurations(one favouring F2 score and one focussing on removing the downhole failures), and got the below results.**<br>

adasyn_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.001/checkpoint_20201112-020820_Epoch_023:<br>
f2 score : &nbsp; 0.895522<br>
Predicted&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;0.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.0<br>
True<br>
0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
11775&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;25<br>
1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
20&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;180<br><br>

smotetomek_data_checkpoint_200_0.1_300_0.2_400_0.3_500_0.4_600_0.5_relu_0.01/checkpoint_20201112-041915_Epoch_039:<br>
f2 score : &nbsp; 0.52356<br>
Predicted&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;0.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.0<br>
True<br>
0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
10890&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;910<br>
1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;200